# BACKUP DE FUNCS IMPORTANTES

## 1. Funcoes Reseva pipeline

In [ ]:
# 7. funçao que MOVE documentos e gera add_log_transaction_entry para df_log_transctions
def move_doc_processed_file(batch_name, src_path, tgt_path):
    
    function = "move_doc_processed_file"
    source_path = src_path
    file = os.path.basename(source_path)
    sub_dir = os.path.join(tgt_path, batch_name)
    destination_path = os.path.join(sub_dir, file)
    document_action = "move_processed_file"
    transaction_detail = (f'document {file} moved by: {function}')
    df_move = pd.DataFrame()
    try:
        document_unique_id = get_document_id_by_file(batch_name, file)
        check_and_create_folder(destination_path)
        shutil.move(source_path, destination_path)
        sucess = True
        move_log = add_log_transaction_entry(document_unique_id, batch_name, file, document_action, src_path, tgt_path, transaction_detail, sucess)
    except Exception as e:
        print(f"Erro ao mover documento: {e}")
        sucess = False
    
    return move_log    

# 8. Função para adicionar um novo registro em df_source
def add_source_entry(batch_name, file_path, file, type, level, parent_unique_id):
    #unique_id = generate_unique_id(type)
    unique_id = generate_unique_id()
    time_now = cron.timenow_pt_BR()   
    file_hash = generate_file_hash(file_path) 
    if level == 1:
        parent_unique_id = unique_id
    data = {
        'Batch': batch_name,
        'Data': time_now,
        'File': file,
        'Type': type,
        'Level': level,
        'Unique_ID': unique_id,
        'Parent_Unique_ID': parent_unique_id,
        'Hash': file_hash,
        'File_Path': file_path
    }
    return data

# 9. Add nova linha para atualizar df_log_transctions
def add_log_transaction_entry(document_unique_id,batch_name, file, document_action, src_path, tgt_path, transaction_detail, sucess=True):

    data_log = {
        'Dt_Time': cron.timenow_pt_BR(),
        'Batch': batch_name,
        'File' : file,
        'Unique_ID': document_unique_id,
        'Action': document_action,
        'Sorce': src_path,
        'Target': tgt_path,
        'Transction_Detail': transaction_detail,
        'Sucess': sucess,    
    }
    
        
    return data_log

## Resrva oficial Aplkicacao

In [ ]:
section = "1. CABECALHO"
label = "1_frame_dados_nf"
pdf_pesquisavel = True
tipo = "frame"
model_map = 'SPA'

coordinates = get_coordinates_filter(pdf_pesquisavel_map=pdf_pesquisavel, model=model_map, tipo=tipo, label=label, section=section)

#coordinates = get_coordinates_filter_pdf_pesquisavel(model_map=model_map, tipo=tipo, label=label, section=section)

x0, y0, x1, y1 = coordinates[0]

print(f'\n3. x0: {x0}, y0: {y0}, x1: {x1}, y1: {y1} \n\n')


In [ ]:
section = "1. CABECALHO"
label = "1_frame_dados_nf"
tipo = "frame"
model_map = 'SPA'

In [ ]:
filtered_boxes_info = frames_nf_v4_df[((frames_nf_v4_df['model'] == model_map) & (frames_nf_v4_df['type'] == tipo) & (frames_nf_v4_df['section_json'] == section) & (frames_nf_v4_df['label'] == label))]

for idx_frame, row_frame in filtered_boxes_info.iterrows():
    extracted_text_box = None
    reference = row_frame['reference']
    x0, y0, x1, y1 = row_frame['x0_p'], row_frame['y0_p'], row_frame['x1_p'], row_frame['y1_p']
    index = idx_frame
    id = row_frame['id']
    
    print(f'\nid: {id} | reference: {reference} x0: {x0}, y0: {y0}, x1: {x1}, y1: {y1} \n\n')


In [ ]:
get_coordinates_filter_pdf_pesquisavel(model_map, tipo, label, section)

In [ ]:
# Esta e o principio da melhor funcao do mundo
def extracao_pipeline(qualquer_df, fase, atividade, status, debug):
    
    doc_info = {}
    resumo = {}
    row_teste_info = []
    time_now = cron.timenow_pt_BR()
    func_fase = fase
    func_atividade = atividade
    func_status = status
    lista_dicts = []
    conf_processo = {}
    lista_conferencia = []
   
    i = 1
    for idx, row in qualquer_df.iterrows():
        dados_iniciais = {}
        row_info = row.to_dict()
        message_erro = []
        # 1. Mapeamento de informacoes do DF
        map_document_unique_id = idx
        map_seq = row['seq']
        map_batch_name = row['batch']
        map_fase_processo = row['fase_processo']
        map_nome_atividade = row['nome_atividade']
        map_status_documento = row['status_documento']
        map_original_file_name = row['original_file_name']
        map_directory = row['directory']
        map_one_page = row['one_page']
        map_palavra_chave = row['palavra_chave']
        map_document_tag = row['document_tag']
        map_action_item = row['action_item']
        map_level = row['level']
        file_path = row['file_path']
        row_info['document_unique_id'] = map_document_unique_id
        # if (map_status_documento != 'NO_PROCESS'):
        # Nivel 1 - Definindo que documentos serao tratados    
        if map_status_documento == 'PREPROCESS_EXTRACT':
            
            # 0. DADOS GERAIS DOCUMENTO
            section = "0. DADOS INICIAIS"
            try:
                valores = {}
                # 0.1. Busco prefeitura, de/para e modelo - se nao achar seta status documento para NO_PROCESS
                valores = processar_dados_iniciais(idx, row, row_info, section, map_directory, map_original_file_name, file_path, debug)
            except Exception as e:
                msg = (f'Erro ao processar_dados_iniciais: {e}')
            finally:
                row_info.update(valores)
            
            map_status_documento_row_info = row_info.get('status_documento')
            
            # Nivel 2 - Definindo que os documentos legiveis serao tratados
            if map_status_documento_row_info == 'PREPROCESS_EXTRACT':
                prefeitura_map = row_info.get('prefeitura')
                pdf_pesquisavel_map = row_info.get('pdf_pesquisavel')
                de_para_map = row_info.get('de_para_pm')
                model_map = row_info.get('model')
                
                
                if not pdf_pesquisavel_map:
                    # 1 XXX NOVO PROCESSO DE TRATAMENTO DE IMAGEM - Convertendo a imagem para numpy array
                    print("irei gerar a imagem_np")
                    imagem_gray, image_resized_name = convert_resize_gray(map_original_file_name, file_path, image_resized_path)
                    imagem_gray_rgb = imagem_gray.convert("RGB")
                    imagem_gray_np = np.array(imagem_gray_rgb)
                    row_info['image_np'] = imagem_gray_np
                
                # 1. CABECALHO
                # try:
                section = "1. CABECALHO"
                valores = {}
                #valores_P = {}
                f_0 = 1
                f_1 = 1
                mapping_method = "frame_&_sframe_field" # significa que as coordenadas estao em frames e os valores dos campos nos sframe_fields
                context_mapping = "data_cabecalho"
                def_replace = True 
                
                if pdf_pesquisavel_map:
                    valores = extrai_cabecalho_PDF_P(idx, row, row_info, section, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path, debug)
                    row_info.update(valores) 
                else:
                    valores = processar_cabecalho_R_PDF(idx, row, row_info, section, mapping_method, context_mapping, pdf_pesquisavel_map, model_map, map_original_file_name, file_path, debug)   
                    row_info.update(valores)
         
                #status_documento_row_info = row_info.get('status_documento')
                action_item_row_info = row_info.get('action_item')
                information_row_info = row_info.get('informations')   
                
                # Nivel 3 - Definindo que os documentos legiveis serao tratados realmente
                if action_item_row_info == 'BREAK_PROCESS':
                    #msg = (f'Processo inicial: {map_batch_name} | {map_original_file_name} | diretorio: {map_directory} - information_row_info: {information_row_info}')
                    print(f'\nINFELIZMENTE - seq: {map_seq} doc: {map_original_file_name} dir: {map_directory} - NAO SERA PROCESSADO  | inf: {information_row_info} \n\n')
                    #row_info['action_item'] = "BREAK_PROCESS"
                    #row_info['informations'] = msg
                    # logging.error(msg)
                    lista_dicts.append(row_info)
                    continue 
                    
                elif action_item_row_info == 'CONTINUE_PROCESS':
                    
                    print(f'\nEBA, BORA CONTINUAR - seq: {map_seq} - proxima section: | PDF Pesquisavel: {pdf_pesquisavel_map} doc: {map_original_file_name} dir: {map_directory} | action_item: {action_item_row_info} | inf: {information_row_info} \n\n')
                    print()
                    print(valores)

                #lista_dicts.append(row_info)
            elif map_status_documento == 'NO_PROCESS':
                msg = (f'Documento sem qualidade para pesquisa inicial: {map_batch_name} | {map_original_file_name} | diretorio: {map_directory}')
                row_info['informations'] = msg  
                row_info['action_item'] = "BREAK_PROCESS"          

        
        
        
        
        
        elif map_status_documento == 'NO_PROCESS':
            msg = (f'Documento nao sera tratado neste escopo: {map_batch_name} | {map_original_file_name} | diretorio: {map_directory}')
            row_info['action_item'] = "NO_PROCESS"    
            row_info['informations'] = msg 
        
        lista_dicts.append(row_info)
        
        i += 1

    logging.info(f'processamento finalizado para: {batch_name}') 
    print(f'processamento de {i} documentos')
    
    novo_df = pd.DataFrame(lista_dicts)
    
    return novo_df

In [ ]:
file_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento/Batch_21/SAO PEDRO DA ALDEIA_PDF_31282023_2257/11756286/NF CRJ PRIMEIRA QUINZENA DE JULHO DE 2023.pdf"

In [ ]:
extracted_txt = pesquisa_prefeitura_pdf_pesquisavel(file_path, debug=True)

In [ ]:
         
       msg = (f'Processo inicial: {map_batch_name} | {map_original_file_name:>25} | diretorio: {map_directory}')
            logging.error(msg)
            row_info['informations'] = msg
            lista_dicts.append(row_info)
            continue
         if debug:
                print(f'\n1. map_status_documento_row_info: {map_status_documento_row_info}\n\n')
                
                           if debug:
                print(f'\n1. apos esecutar processar_dados_iniciais: valores:\n\n{valores}\n\n')
                
                
                print(f'\nseq: {map_seq} | PDF Pesquisavel: {pdf_pesquisavel_map} doc: {map_original_file_name} dir: {map_directory} valores: {valores}\n')
                
                
                
                            print(f'\n1. processo inicial: {map_status_documento_row_info} - doc: {map_original_file_name} | diretorio: {map_directory} \n')
            'NO_PROCESS':
                msg = (f'Processo inicial: {batch_name} | {map_original_file_name:>25} | diretorio: {map_directory}')
                logging.error(msg)
                row_info['informations'] = msg
                lista_dicts.append(row_info)
                continue

In [ ]:
def extrai_cabecalho_PDF_P(idx, row, row_info, pdf_pesquisavel_map, de_para_pm, model_map, f_0, f_1, original_file_name, file_path, debug):
    
    nf_data_cabecalho = {}
    section = "1 - CABECALHO"
    pdf_pesquisavel_map = True
    lista_erros = []
    batch_name = row['batch']
    print(f'\n\n2. dentro da funçao extrai_cabecalho_PDF: batch_name: {batch_name}\n\n')
    
    label = "1_frame_dados_nf"

    pdf_document = fitz.open(file_path)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]    
    tipo = "frame"

    coordinates = get_coordinates_filter(pdf_pesquisavel_map=pdf_pesquisavel_map, model=model_map, tipo=tipo, label=label, section=section)
    x0, y0, x1, y1 = coordinates[0]
    #print(label)
    #print(x0,x1,y0,y1)
    y0 = y0 * f_0
    y1 = y1 * f_1
    text = page.get_text("text", clip=(x0, y0, x1, y1))
    nf_data_cabecalho = novaextra.extract_fields_cabecalho(idx, row, row_info, text, original_file_name, debug)
    
    
    pdf_document.close()
    return nf_data_cabecalho



    return data_extrated_prefeitura

In [ ]:
# Esta e o principio da melhor funcao do mundo
def extracao_pipeline(qualquer_df, fase, atividade, status, debug):
    
    doc_info = {}
    resumo = {}
    row_teste_info = []
    time_now = cron.timenow_pt_BR()
    func_fase = fase
    func_atividade = atividade
    func_status = status
    lista_dicts = []
    conf_processo = {}
    lista_conferencia = []
   
    i = 1
    for idx, row in qualquer_df.iterrows():
        dados_iniciais = {}
        row_info = row.to_dict()
        message_erro = []
        # 1. Mapeamento de informacoes do DF
        map_document_unique_id = idx
        map_seq = row['seq']
        map_batch_name = row['batch']
        map_fase_processo = row['fase_processo']
        map_nome_atividade = row['nome_atividade']
        map_status_documento = row['status_documento']
        map_original_file_name = row['original_file_name']
        map_directory = row['directory']
        map_one_page = row['one_page']
        map_palavra_chave = row['palavra_chave']
        map_document_tag = row['document_tag']
        map_action_item = row['action_item']
        map_level = row['level']
        file_path = row['file_path']
        # if (map_status_documento != 'NO_PROCESS'):
            #1. Buscar Prefeitura, de/para e modelo
        if map_status_documento == 'NO_PROCESS':
            msg = (f'Processo inicial: {batch_name} | {map_original_file_name:>25} | diretorio: {map_directory}')
            logging.error(msg)
            row_info['informations'] = msg
            lista_dicts.append(row_info)
            continue
        
        elif map_status_documento == 'PREPROCESS_EXTRACT':   
            
            valores = {}
            valores = processar_dados_iniciais(idx, row, row_info, map_directory, map_original_file_name, file_path, debug)
            if debug:
                print(f'\n1. apos esecutar processar_dados_iniciais: valores:\n\n{valores}\n\n')
            if valores:
                row_info.update(valores)
            map_status_documento_row_info = row_info.get('status_documento')
            if debug:
                print(f'\n1. map_status_documento_row_info: {map_status_documento_row_info}\n\n')
            
            print(f'\n1. processo inicial: {map_status_documento_row_info} - doc: {map_original_file_name} | diretorio: {map_directory} \n')
            if map_status_documento_row_info == 'NO_PROCESS':
                msg = (f'Processo inicial: {batch_name} | {map_original_file_name:>25} | diretorio: {map_directory}')
                logging.error(msg)
                row_info['informations'] = msg
                lista_dicts.append(row_info)
                continue
            
            elif map_status_documento_row_info == 'PREPROCESS_EXTRACT':
                # 0. DADOS GERAIS DOCUMENTO
                prefeitura_map = row_info['prefeitura']
                pdf_pesquisavel_map = valores['pdf_pesquisavel']
                de_para_map = valores['de_para_pm']
                model_map = valores['model']
                #cnpj_map = dados_iniciais['cnpj']
                if debug:
                    print(f'\n1. map_original_file_name: {map_original_file_name} | pdf_pesquisavel: {pdf_pesquisavel_map} | diretorio: {map_directory} | de_para_map: {de_para_map} | model: {model_map} | PM: {prefeitura_map}\n')
                
                row_info['document_unique_id'] = map_document_unique_id
                if not pdf_pesquisavel_map:
                    # 1 XXX NOVO PROCESSO DE TRATAMENTO DE IMAGEM - Convertendo a imagem para numpy array
                    print("irei gerar a imagem_np")
                    imagem_gray, image_resized_name = convert_resize_gray(map_original_file_name, file_path, image_resized_path)
                    imagem_gray_rgb = imagem_gray.convert("RGB")
                    imagem_gray_np = np.array(imagem_gray_rgb)
                    row_info['image_np'] = imagem_gray_np

                # 1. CABECALHO
                section = "1. CABECALHO"
                valores = {}
                f_0 = 1
                f_1 = 1
                mapping_method = "frame_&_sframe_field" # significa que as coordenadas estao em frames e os valores dos campos nos sframe_fields
                context_mapping = "data_cabecalho"
                def_replace = True
                
                if pdf_pesquisavel_map:
                    valores = extrai_cabecalho_PDF_P(idx, row, row_info, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path, debug)
                    print(f'\nseq: {map_seq} | PDF Pesquisavel: {pdf_pesquisavel_map} doc: {map_original_file_name} dir: {map_directory} valores: {valores}\n')    
                else:
                    valores = processar_cabecalho_R_PDF(idx, row, row_info, mapping_method, context_mapping, pdf_pesquisavel_map, model_map, map_original_file_name, file_path, debug)
                    
                print(f'\nseq: {map_seq} | PDF Pesquisavel: {pdf_pesquisavel_map} doc: {map_original_file_name} dir: {map_directory} valores:\n\n{valores}\n\n\n')
                
                if valores:
                    row_info.update(valores)
                       
                
                map_status_documento_row_info = row_info.get('status_documento')
                
                print(f'\n2. Cabecalho: {map_status_documento_row_info} - doc: {map_original_file_name} | diretorio: {map_directory} \n')
                
                if map_status_documento_row_info == 'NO_PROCESS':
                    msg = (f'Processo inicial: {batch_name} | {map_original_file_name:>25} | diretorio: {map_directory}')
                    logging.error(msg)
                    row_info['informations'] = msg
                    #lista_dicts.append(row_info)
                    continue
                
                elif map_status_documento_row_info == 'PREPROCESS_EXTRACT':
                    
                    msg = (f'secao: {section:>15} processada para: {map_original_file_name} - diretorio: {map_directory}')
                    # if debug:
                    #     print(msg)
                    # logging.info(msg)  

                    #     #if map_status_documento_row_info == 'PREPROCESS_EXTRACT':        
                    #     # map_status_documento_valores = valores['status_documento']
                    #     # if map_status_documento_valores:
                            
                    # guarda_texto_doc = {}
                    # guarda_texto_doc, linhas = cria_guarda_doc_ref_R_PDF(idx, row, de_para_map, model_map, map_original_file_name, file_path, image_resized_path, debug)
            
                    # # 2. PRESTADOR DE SERVIÇO
                    # section = "2. PRESTADOR DE SERVIÇO"
                    # valores = {}
                    # erros_prestador = {}
                    # data_tomador = {}
                    # f_0 = 1
                    # f_1 = 1
                    # if pdf_pesquisavel_map:
                    #     valores = extrai_prestador_PDF_P(row, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path, debug)
                    # else:
                    #     valores = extrai_prestador_R_PDF(idx, row, row_info, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path, debug)
                    
                    # if not isinstance(valores, dict):
                    #     msg_erro = (f"\nErro na linha {idx}: 'valores' não é um dicionário. Tipo: {type(valores)}, Valor: {valores}")
                    # else:
                    #     row_info.update(valores)
                        
                    # msg = (f'secao: {section:>15} processada para: {map_original_file_name} - diretorio: {map_directory}')
                    # if debug:
                    #     print(msg)
                    # logging.info(msg)                                                    

                    # # 3. TOMADOR DE SERVIÇO
                    # section = "3. TOMADOR DE SERVIÇO"
                    # valores = {}
                    # erros = []
                    # data_tomador = {}
                    # f_0 = 1
                    # f_1 = 1.01
                    
                    # if pdf_pesquisavel_map:
                    #     valores = extrai_tomador_PDF_P(row, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path, debug)
                    # else:   
                    #     valores = extrai_tomador_R_PDF(idx, row, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path, debug)
                        
                    # if not isinstance(valores, dict):
                    #     print(f"\nErro na linha {idx}: 'valores' não é um dicionário. Tipo: {type(valores)}, Valor: {valores}")
                    # else:
                    #     row_info.update(valores)
                        
                    # msg = (f'secao: {section:>15} processada para: {map_original_file_name} - diretorio: {map_directory}')
                    # if debug:
                    #     print(msg)
                    # logging.info(msg)    
                        
                        
                    
                    # # 4. DESCRIMINACAO DOS SERVIÇOS
                    # section = "4. DESCRIMINACAO DOS SERVIÇOS"
                    # valores = {}
                    # nf_data_servico = {} 
                    # f_0 = 1
                    # f_1 = 1
                    
                    # if pdf_pesquisavel_map:
                    #     nf_data_servico = processar_servicos_pdf_pesquisavel(row, pdf_pesquisavel_map, model_map, map_original_file_name, file_path)
                    # else:
                    #     label = "discriminacao_servicos"
                    #     tipo = "field_box"
                    #     def_replace = True
                        
                    #     # ItSs  working
                    #     texto_extraido = extracao_documento_R_PDF(idx, row, guarda_texto_doc, section, tipo, label, de_para_map, model_map, def_replace, map_original_file_name)
                    #     row_info[label] = texto_extraido
                        
                    # msg = (f'secao: {section:>15} processada para: {map_original_file_name} - diretorio: {map_directory}')
                    # if debug:
                    #     print(msg)
                    # logging.info(msg)     


                    # try:
                    #     texto_extraido = nf_data_servico['discriminacao_servicos'] 
                    #     row_info['discriminacao_servicos'] = texto_extraido 
                    # except Exception as e:
                    #     msg = (f"doc: {map_original_file_name} | {e}")
                    #     discrimanacao_servico = "Descricao nao encontrada"
                    #     row_info['discriminacao_servicos'] = texto_extraido

                    
                    # # 5. VALOR TOTAL
                    # section = "5. VALOR TOTAL"
                    # valores = {}
                    # if pdf_pesquisavel_map:
                    #     valores = processar_valor_total_PDF_P(row, pdf_pesquisavel_map, model_map, map_original_file_name, file_path, debug)
                    # else:
                    #     label = "valor_total_nota"
                    #     tipo = "field_box"
                    #     def_replace = True
                    #     texto_extraido = extracao_documento_R_PDF(idx, row, guarda_texto_doc, section, tipo, label, de_para_map, model_map, def_replace, map_original_file_name, debug)
                    #     if texto_extraido: 
                    #         valor_total_match = re.search(r'R\$ ([\d,.]+)', texto_extraido)
                    #         if valor_total_match:
                    #             valor_total_sem_formatacao = valor_total_match.group(1).replace('.', '').replace(',', '.')
                    #             try:
                    #                 valores['valor_total_nota'] = float(valor_total_sem_formatacao)
                    #             except Exception as e:
                    #                 msg = (f'Processo inicial: {batch_name} | {map_original_file_name:>25} | diretorio: {map_directory} | {e}')
                    #                 valores['valor_total_nota'] = 0.0
                    #                 logging.error(f" {batch_name} |  doc: {original_file_name:>25} | setion:{section:20} | item: {key:>20} | erro na extracaçao | file_path: {file_path:>40} ")
                    
                    # if not isinstance(valores, dict):
                    #     msg_erro = (f"\nErro na linha {idx}: 'valores' não é um dicionário. Tipo: {type(valores)}, Valor: {valores}")
                    # else:
                    #     row_info.update(valores)
                        
                    # msg = (f'secao: {section:>15} processada para: {map_original_file_name} - diretorio: {map_directory}')
                    # if debug:
                    #     print(msg)
                    # logging.info(msg)   

                    
                    # # 6. CNAE e Item da Lista de Serviços 
                    # section = "6. CNAE e Item da Lista de Serviços"
                    # f_0_cnae = 0.95
                    # f_1_cnae = 1.15
                    # f_0_it = 0.95     #0.95
                    # f_1_it = 1.15    # 1
                    # if pdf_pesquisavel_map:
                    #     cnae_value, item_servico_value = extrai_consiste_cnae_PDF_P(row, pdf_pesquisavel_map, de_para_map, model_map, f_0_cnae, f_1_cnae, f_0_it, f_1_it, map_original_file_name, file_path, debug)
                    # else:
                        
                    #     tipo = "sframe_field"
                    #     def_replace = True                  
                    #     label = "cnae"
                    #     cnae_value = extracao_documento_R_PDF(idx, row, guarda_texto_doc, section, tipo, label, de_para_map, model_map, def_replace, map_original_file_name)
                    #     if cnae_value:
                    #         row_info[label] = cnae_value
                            
                    #     label = "item_lista_servicos"
                    #     item_lista_value = extracao_documento_R_PDF(idx, row, guarda_texto_doc, section, tipo, label, de_para_map, model_map, def_replace, map_original_file_name)
                    #     if item_lista_value:
                    #         row_info[label] = cnae_value
                    #         #print(f'cnae: {cnae_value} | item_lista_servicos: {item_lista_value}')
                    
                    # msg = (f'secao: {section:>15} processada para: {map_original_file_name} - diretorio: {map_directory}')
                    # if debug:
                    #     print(msg)
                    # logging.info(msg)         

                    # # 7. VALORES E IMPOSTOS
                    # section = "7. VALORES E IMPOSTOS"
                    # valores = {}
                    # nf_data_valores = {}
                    # lista_impostos = []
                    # f_0 = 1
                    # f_1 = 1
                    # if pdf_pesquisavel_map:
                    #     valores = extrai_valores_impostos_PDF_P(row, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path)
                    #     row_info.update(valores)
                    
                    # else:
                    #     tipo = "field_box"
                    #     father_value = "5_frame_valores_impostos"
                    #     valores = extracao_impostos_R_PDF(section, tipo, father_value, de_para_map, model_map, map_original_file_name, file_path)
                    #     #print(valores)
                    #     row_info.update(valores)
                        
                    # msg = (f'secao: {section:>15} processada para: {map_original_file_name} - diretorio: {map_directory}')
                    # if debug:
                    #     print(msg)
                    # logging.info(msg)   
                        
                    # # 8. DADOS COMPLEMENTARES
                    # section = '8. DADOS COMPLEMENTARES'
                    # nf_data_dados_complementares = {}
                    # f_0 = 1
                    # f_1 = 1
                    # if pdf_pesquisavel_map:
                    #     nf_data_valores = extrai_dados_complementares_PDF_P(row, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path)
                    # else:
                        
                    #     tipo = "field_box"
                    #     father_value = "6_section_inf_complementares_criticas"  
                    #     valores = extracao_complementares_R_PDF(row, section, tipo, father_value, de_para_map, model_map, map_original_file_name, file_path)
                    #     if valores:
                    #         row_info.update(valores)    
                    # try:
                    #     row_info['dados_complementares'] = nf_data_dados_complementares['dados_complementares']
                    # except Exception as e:
                    #     msg = (f"doc: {map_original_file_name} | {e}")
                    #     row_info['dados_complementares'] = None
                    
                    # msg = (f'secao: {section:>15} processada para: {map_original_file_name} - diretorio: {map_directory}')
                    # if debug:
                    #     print(msg)
                    # logging.info(msg)     
                    
                    # # 9. OUTRAS INFORMAÇOES / CRITICAS
                    # section = "9. OUTRAS INFORMAÇOES / CRITICAS"
                    # tipo = "field_box"
                    # father_value = "5_frame_inf_criticas"
                    # valores = {} 
                    # nf_data_outras_informacoes = {}
                    # f_0 = 1
                    # f_1 = 1
                    # if pdf_pesquisavel_map:
                    #     valores = extrai_outras_informacoes_PDF_P(row, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path)
                    #     if valores:
                    #         row_info.update(valores)
                    # else:
                    #     section = "9. OUTRAS INFORMAÇOES / CRITICAS"
                    #     tipo = "field_box"
                    #     father_value = "5_frame_inf_criticas"
                    #     valores = extracao_inforacoes_criticas_R_PDF(section, tipo, father_value, de_para_map, model_map, map_original_file_name, file_path)
                    #     if valores:
                    #         row_info.update(valores)
                            
                    # msg = (f'secao: {section:>15} processada para: {map_original_file_name} - diretorio: {map_directory}')
                    # if debug:
                    #     print(msg)
                    # logging.info(msg)          
                            
                    
                    # # 10. OBSERVACOES  
                    # section = "10. OBSERVACOES"  
                    # data_observacao = {}
                    # valores = {}
                    # f_0 = 0.9
                    # f_1 = 1.1
                    # if pdf_pesquisavel_map:
                    #     valores = extrai_outras_informacoes_PDF_P(row, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path)
                    #     if valores:
                    #         row_info.update(valores)
                    # else:
                    #     section = '10. OBSERVACOES'
                    #     tipo = "field_box"
                    #     father_value = "6_section_inf_complementares_criticas"  
                    #     valores = extracao_observacao_R_PDF(row, section, tipo, father_value, de_para_map, model_map, map_original_file_name, file_path) 
                    #     if valores:
                    #         row_info.update(valores)
                    
                    # msg = (f'secao: {section:>15} processada para: {map_original_file_name} - diretorio: {map_directory}')
                    # if debug:
                    #     print(msg)
                    logging.info(msg)            
                    lista_dicts.append(row_info)

        i += 1
        
    logging.info(f'processamento finalizado para: {batch_name}') 
    
    novo_df = pd.DataFrame(lista_dicts)
    return novo_df

In [ ]:
# Ajustando DF para analises
df.set_index('document_unique_id', inplace=True)
ordem_status = ['PREPROCESS_EXTRACT', 'NO_PROCESS', 'root_analise']
df['status_documento'] = pd.Categorical(df['status_documento'], categories=ordem_status, ordered=True)
df.sort_values(by=['status_documento', 'seq'], ascending=[True, True], inplace=True)

In [ ]:
def extracao_pipeline(qualquer_df, fase, atividade, status, debug):
    
    doc_info = {}
    resumo = {}
    row_teste_info = []

    i = 1
    for idx, row in qualquer_df.iterrows():
        dados_iniciais = {}
        row_info = row.to_dict()
        message_erro = []
        # 1. Mapeamento de informacoes do DF
        map_document_unique_id = idx
        map_seq = row['seq']
        map_batch_name = row['batch']
        map_fase_processo = row['fase_processo']
    
        if map_status_documento == 'PREPROCESS_EXTRACT':
            
            valores = {}
            valores = processar_dados_iniciais(idx, row, row_info, map_directory, map_original_file_name, file_path, debug)
            if debug:
                print(f'\n1. apos esecutar processar_dados_iniciais: valores:\n\n{valores}\n\n')
            if valores:
                row_info.update(valores)
            map_status_documento_row_info = row_info.get('status_documento')
            
            if map_status_documento_row_info == 'NO_PROCESS':
                msg = (f'Processo inicial: {batch_name} | {map_original_file_name:>25} | diretorio: {map_directory}')
                logging.error(msg)
                row_info['informations'] = msg
                lista_dicts.append(row_info)
                return
            
            elif map_status_documento_row_info == 'PREPROCESS_EXTRACT':
                # 0. DADOS GERAIS DOCUMENTO
                prefeitura_map = row_info['prefeitura']
                pdf_pesquisavel_map = valores['pdf_pesquisavel']
                de_para_map = valores['de_para_pm']
                model_map = valores['model']
                #cnpj_map = dados_iniciais['cnpj']
                row_info['document_unique_id'] = map_document_unique_id
                
                if not pdf_pesquisavel_map:
                    # 1 XXX NOVO PROCESSO DE TRATAMENTO DE IMAGEM - Convertendo a imagem para numpy array
                    print("irei gerar a imagem_np")
                    imagem_gray, image_resized_name = convert_resize_gray(map_original_file_name, file_path, image_resized_path)
                    imagem_gray_rgb = imagem_gray.convert("RGB")
                    imagem_gray_np = np.array(imagem_gray_rgb)
                    row_info['image_np'] = imagem_gray_np
                 
                
                # 1. CABECALHO
                section = "1. CABECALHO"
                valores = {}
                f_0 = 1
                f_1 = 1
                if pdf_pesquisavel_map:
                    valores = extrai_cabecalho_PDF_P(row, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path, debug)
                    if valores:
                        row_info.update(valores)
                else:
                    mapping_method = "frame_&_sframe_field" # significa que as coordenadas estao em frames e os valores dos campos nos sframe_fields
                    context_mapping = "data_cabecalho"
                    def_replace = True
                    valores = processar_cabecalho_R_PDF(idx, row, row_info, mapping_method, context_mapping, pdf_pesquisavel_map, model_map, map_original_file_name, file_path, debug)
                    if valores:
                        row_info.update(valores)
                        map_status_documento_row_info = row_info['status_documento']
                    
                    msg = (f'secao: {section:>15} processada para: {map_original_file_name} - diretorio: {map_directory}')
                    if debug:
                        print(msg)
                    logging.info(msg)  

           
                guarda_texto_doc = {}
                guarda_texto_doc, linhas = cria_guarda_doc_ref_R_PDF(idx, row, de_para_map, model_map, map_original_file_name, file_path, image_resized_path, debug)
        
                # 2. PRESTADOR DE SERVIÇO
                section = "2. PRESTADOR DE SERVIÇO"
                valores = {}
                f_0 = 1
                f_1 = 1
                if pdf_pesquisavel_map:
                    valores = extrai_prestador_PDF_P(row, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path, debug)
                else:
                    valores = extrai_prestador_R_PDF(idx, row, row_info, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path, debug)
       
                # 3. TOMADOR DE SERVIÇO
                section = "3. TOMADOR DE SERVIÇO"
                valores = {}
                erros = []
              
                
                # 4. DESCRIMINACAO DOS SERVIÇOS
                section = "4. DESCRIMINACAO DOS SERVIÇOS"
                valores = {}
                nf_data_servico = {} 
                f_0 = 1
                f_1 = 1
            
                # 6. CNAE e Item da Lista de Serviços 
                section = "6. CNAE e Item da Lista de Serviços"
          

                # 7. VALORES E IMPOSTOS
                section = "7. VALORES E IMPOSTOS"
                valores = {}
                nf_data_valores = {}
          
                # 8. DADOS COMPLEMENTARES
                section = '8. DADOS COMPLEMENTARES'
                nf_data_dados_complementares = {}
          
                # 9. OUTRAS INFORMAÇOES / CRITICAS
                section = "9. OUTRAS INFORMAÇOES / CRITICAS"
                tipo = "field_box"
       
                # 10. OBSERVACOES  
                section = "10. OBSERVACOES"  
                data_observacao = {}
        
   
                            
                lista_dicts.append(row_info)

        i += 1
        

    novo_df = pd.DataFrame(lista_dicts)
    logging.info(f'processamento finalizado para: {batch_name}')  
     
    return novo_df

In [ ]:
                    # if debug:
                    #     x0 = 0
                    #     y0 = 0
                    #     plt.figure(figsize=(25, 25))
                    #     plt.imshow(imagem_gray_np)
                    #     plt.axis('off')  # Desativa os eixos para uma visualização mais limpa
                    #     plt.text(x0 + 1, y0 + 10, map_original_file_name, color='black', fontsize=20)
                    #     #plt.text(original_file_name, color='black', fontsize=7)
                    #     plt.show()
                    # row_info['image_np'] = imagem_gray_np
                    # conf_processo['image_np'] = imagem_gray_np

In [ ]:
                    #if valores:
                        #print(f'\ncabecalho PDF_Pesquisavel: {map_original_file_name} - diretorio: {map_directory} valores:\n\n{valores}\n\n\n')
                        #row_info.update(valores)

In [ ]:

def get_coordinates_filter_R_PDF(model_map, tipo, label, section):
    
    row_frame = filtrar_df(frames_nf_v4_df, model=model_map, type=tipo, label=label, section_json=section)
    
    # Verificando se row_frame não está vazio
    if not row_frame.empty:
        # Acessando a primeira linha do DataFrame filtrado e depois acessando as colunas
        coodinates = [(row_frame.iloc[0]['x0'], row_frame.iloc[0]['y0'], row_frame.iloc[0]['x1'], row_frame.iloc[0]['y1'])]
    else:
        # Retornando uma tupla de valores NaN se o DataFrame filtrado estiver vazio
        coodinates = [(float('nan'), float('nan'), float('nan'), float('nan'))]
    
    return coodinates




def get_coordinates_filter_pdf_pesquisavel(model_map, tipo, label, section):
    
    row_frame = filtrar_df(frames_nf_v4_df, model=model_map, type=tipo, label=label, section_json=section)
    
    # Verificando se row_frame não está vazio
    if not row_frame.empty:
        # Acessando a primeira linha do DataFrame filtrado e depois acessando as colunas
        coodinates = [(row_frame.iloc[0]['x0_p'], row_frame.iloc[0]['y0_p'], row_frame.iloc[0]['x1_p'], row_frame.iloc[0]['y1_p'])]
    else:
        # Retornando uma tupla de valores NaN se o DataFrame filtrado estiver vazio
        coodinates = [(float('nan'), float('nan'), float('nan'), float('nan'))]
    
    return coodinates

In [ ]:
# 0. XXX Nova funçao de conversao de PDF para imagem tam 2X
def convertResize2(idx, row, original_file_name, file_path, image_resized_path):
    
    
    name_image = conv_filename_no_ext(original_file_name)
    
    image_resized_name = os.path.join(f'{image_resized_path}/{str(name_image)}.jpg')
    #print(f'image_resized_name: {image_resized_name}\n')
    # 3. Conversao para imagem
    pages = convert_from_path(file_path, 500, poppler_path=poppler_path)
    
    # 4. Verifica se ha mais que uma pagina
    if len(pages) > 1:
        raise ValueError("Erro, documento com mais de uma página")
    else:
        # 5. Iterar pelas páginas e redimensionar
        resized_pages = []
        for page in pages:
            resized_page = page.resize((4134, 5846))
            resized_pages.append(resized_page)
            
        resized_pages[0].save(image_resized_name, 'JPEG')
        
    return  image_resized_name

In [ ]:
# Modelo de apresentaçao da imagem
x0 = 0
y0 = 0
plt.figure(figsize=(25, 25))
plt.imshow(imagem_gray_np)
plt.axis('off')  # Desativa os eixos para uma visualização mais limpa
plt.text(x0 + 1, y0 + 10,original_file_name, color='black', fontsize=20)
#plt.text(original_file_name, color='black', fontsize=7)
plt.show()

In [ ]:
if mapping_method == "frame_&_sframe_field":
    tipo_4_coordinates = "frame"
    tipo_4_filter = "sframe_field"
coordinates = get_coordinates_filter_by_context(pdf_pesquisavel_map, model, context_mapping, tipo_4_coordinates)
x0, y0, x1, y1 = coordinates[0]

In [ ]:
def filtrar_df(df, **kwargs):
    query = " & ".join(f"{key} == @kwargs['{key}']" for key in kwargs)
    result = df.query(query)
    return result

# 11. Pesquiso Unique_ID por file
def get_document_id_by_file(batch, file):
    
    result = filtrar_df(df_id_relations, Batch=batch, File=file)
    document_unique_id = result['Unique_ID'].values[0]
    
    return document_unique_id

In [ ]:
df_conf0.columns

In [ ]:
df_conf0

In [ ]:
original_file_name = "Doria Marinho 0297 Raquel.pdf"

In [ ]:
result = filtrar_df(df_conf0, original_file_name=original_file_name)

In [ ]:
result

In [ ]:
# Modelo de apresentaçao da imagem
x0 = 0
y0 = 0
plt.figure(figsize=(25, 25))
plt.imshow(result['image_np'].values[0])
plt.axis('off')  # Desativa os eixos para uma visualização mais limpa
plt.text(x0 + 1, y0 + 10,original_file_name, color='black', fontsize=20)
#plt.text(original_file_name, color='black', fontsize=7)
plt.show()

In [ ]:

image_np = result['image_np'].values[0]

In [ ]:
image_np

In [ ]:
x0 = int(x0)
y0 = int(y0)
x1 = int(x1)    
y1 = int(y1)

In [ ]:
cropped_image_np = imagem_gray_np[y0:y1, x0:x1]

In [ ]:
# Modelo de apresentaçao da imagem
x0 = 0
y0 = 0
plt.figure(figsize=(5, 5))
plt.imshow(cropped_image_np)
plt.axis('off')  # Desativa os eixos para uma visualização mais limpa
#plt.text(x0 + 1, y0 + 10,original_file_name, color='black', fontsize=20)
#plt.text(original_file_name, color='black', fontsize=7)
plt.show()

In [ ]:
cropped_image_np

In [ ]:
boxes = frames_nf_v4_df[frames_nf_v4_df['type'] == 'field_box']

In [ ]:
boxes_info = {}
i = 1
for idx, row in boxes.iterrows():
    x0 = int(row['x0'])
    y0 = int(row['y0'])
    x1 = int(row['x1'])
    y1 = int(row['y1'])
    cropped_image_np = imagem_gray_np[y0:y1, x0:x1]
    
    boxes_info[f'box_{i}'] = {
        'coordinates': (x0, y0, x1, y1),
        'image': cropped_image_np,
        # ... qualquer outra informação que você deseja armazenar
    }
    i += 1


In [ ]:
boxes_info['box_1']['coordinates']

In [ ]:
boxes_info['box_1']['image']

In [ ]:
for box in boxes_info:
    x0, y0, x1, y1 = boxes_info[box]['coordinates']
    image = boxes_info[box]['image']
    # plt.figure(figsize=(25, 25))
    plt.imshow(image)
    plt.axis('off')  # Desativa os eixos para uma visualização mais limpa
    plt.show()

In [ ]:
i = 8

x0 = 0
y0 = 0
plt.figure(figsize=(10, 10))
plt.imshow(boxes_info[f'box_{i}']['image'])
plt.text(x0 , y0,boxes_info[f'box_{i}']['coordinates'], color='green', fontsize=7)
plt.axis('off')  # Desativa os eixos para uma visualização mais limpa
plt.show()


In [ ]:
# Inicialize um dicionário vazio para armazenar as informações dos "boxes"
boxes_info = {}

# Suponha que você está em um loop onde está processando vários "boxes"
for i, box in enumerate(boxes):
    # Obtenha as coordenadas do "box"
    x0, y0, x1, y1 = coordinates[0] 
    # Corte a área do "box" da imagem original
    cropped_image = imagem_gray_np[y0:y1, x0:x1]
    
    # Armazene as informações do "box" no dicionário
    boxes_info[f'box_{i}'] = {
        'coordinates': (x0, y0, x1, y1),
        'image': cropped_image,
        # ... qualquer outra informação que você deseja armazenar
    }

# Agora, `boxes_info` contém informações detalhadas sobre cada "box"


In [ ]:
section = "10. OBSERVACOES"
tipo = "field_box"
father_value = "5_frame_inf_criticas"
marcador_ini = "Observação:"

In [ ]:
# Esta e o principio da melhor funcao do mundo - SIM ELA E MAS PRECISA FICAR AQUI?
def extracao_pipeline(qualquer_df, fase, atividade, status):
    
    linhas_analise = []
    bloco_1_list = []
    bloco_2_list = []
    bloco_3_list = []
    imagens_list = []  
    new_data = [] 
    
    pre_processo = ['nro_nota', 'competencia', 'dt_hr_emissao', 'codigo_verificacao', 'ajustar_nome', 'split_paginas', 'ajustar_imagem', 'buscar_nome_prefeitura', 'enviar_canceladas', 'enviar_listagens']
    
    time_now = cron.timenow_pt_BR()
    
    fase_processo_atual = fase
    atividade_processo_atual = atividade
    status_documento_atual = status
    
    i = 1
    for idx, row in qualquer_df.iterrows():
        message_erro = []
        
        # 1. Mapeamento de informacoes do DF
        document_unique_id = idx
        seq_df = row['seq']
        batch_name = row['batch']
        fase_processo = row['fase_processo']
        nome_atividade = row['nome_atividade']
        status_documento = row['status_documento']
        original_file_name = row['original_file_name']
        file_directory = row['directory']
        level = row['level']
        d_type = row['level']
        document_type = row['document_type']
        pdf_pesquisavel = row['pdf_pesquisavel']
        one_page_doc = row['one_page']
        modelo = row['modelo']
        
        prefeitura = row['prefeitura']
        
        parent_document_unique_id = row['parent_document_unique_id']
        file_path = row['file_path']
        file_hash = row['file_hash']
        
        # 2. Busca modelo
        if (not status_documento == 'NAO_PROCESSAR') or (not document_type == 'outros') or (not one_page_doc == True):
            
            if atividade_processo_atual == 'extracao_prestador':
            #print(f' 1 - seq: {seq_df} | file: {original_file_name} |status_documento: {status_documento} pdf_pesquisavel: {pdf_pesquisavel}\n')
                if status_documento == status_documento_atual:
         
                    print(f'seq_df: {seq_df} status_documento: {status_documento} | modelo: {modelo:>20} | file: {original_file_name:>40} | prefeitura: {prefeitura:>55} | {pdf_pesquisavel}\n')
                    
                    result_list = []
                    
                    dfcnpj_prestador = {}
                    dfincricao_prestador = {}
                    dfdados_prestador = {}
                        
                    doc2convert = original_file_name
                    section = "2. PRESTADOR DE SERVIÇO"
                    modelo = 'SAO_PEDRO_SUPERMIX'
                    f_tipo = 'frame'
                    
                    dfcnpj_prestador, dfincricao_prestador, dfdados_prestador, textoextraido  = processar_dados_dados_documentos(row, original_file_name, file_path, pdf_pesquisavel, section, modelo, f_tipo)
                    
                    print(f'\n {dfcnpj_prestador}\n{dfincricao_prestador}\n{dfdados_prestador}\n')
                    
               
      


                i += 1
              
    return textoextraido

In [ ]:
texto_valores = linhas
# Exemplo de uso
marcador1 = 'VALOR SERVIÇOS'  # SAO PEDRO DA ALDEIA
marcador2 = 'DADOS COMPLEMENTARES'
lista_val = encontrar_valores3(texto_valores, marcador1, marcador2)


print(f'\nlista_val: {lista_val}  -->  {len(lista_val)}\n')

In [ ]:
# Nota 1
file_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento/Batch_20/MAGE_PDF_31282023_2254/10597/NFS.299 07.2023 FLOC.pdf"
doc2convert = "NFS.299 07.2023 FLOC.pdf"

image_2work, image_path = convertResize(doc2convert, file_path, image_resized_path)
imagem_gray = image_2work.convert('L')
texto_doc = (pytesseract.image_to_string(imagem_gray, lang='por'))
text_splited = texto_doc.split('\n')
texto_ref = [x for x in text_splited if x.strip()]
texto_ref

In [ ]:
texto_valores = texto_ref

# Exemplo de uso
marcador1 = 'VALOR SERVIÇOS:'  # SAO PEDRO DA ALDEIA
marcador2 = 'LOCAL INCIDÊNCIA'
lista_val = encontrar_valores3(texto_valores, marcador1, marcador2)


print(f'\nlista_val: {lista_val}  -->  {len(lista_val)}\n')

In [ ]:
# Nota 2
file_path = "pipeline_extracao_documentos/7_Avaliar_processamento/Batch_2/SAO PEDRO DA ALDEIA_PDF_17282023_1532/11282686/2023251 - RAZÃO ORGANIZAÇÃO CONTABIL - LTDA.pdf"
doc2convert = "2023251 - RAZÃO ORGANIZAÇÃO CONTABIL - LTDA.pdf"

image_2work, image_path = convertResize(doc2convert, file_path, image_resized_path)
imagem_gray = image_2work.convert('L')
texto = (pytesseract.image_to_string(imagem_gray, lang='por'))
texto

In [ ]:
linhas = [linha for linha in texto.split('\n') if linha.strip()]
linhas

In [ ]:
texto_valores = linhas

# Exemplo de uso
marcador1 = 'VALOR SERVIÇOS:'  # SAO PEDRO DA ALDEIA
marcador2 = 'LOCAL INCIDÊNCIA'
lista_val = encontrar_valores2(texto_valores, marcador1, marcador2)
lista_val

print(f'\nlista_val: {lista_val}  -->  {len(lista_val)}\n')

In [ ]:
# Nota 3
file_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento/Batch_20/MAGE_PDF_31282023_2254/1004403/NF 11 IGREJA EVANGÉLICA BATISTA ETERNA VIDA.pdf"
doc2convert = "NF 11 IGREJA EVANGÉLICA BATISTA ETERNA VIDA.pdf"

image_2work, image_path = convertResize(doc2convert, file_path, image_resized_path)
imagem_gray = image_2work.convert('L')
texto_doc = (pytesseract.image_to_string(imagem_gray, lang='por'))
text_splited = texto_doc.split('\n')
texto_refNota3 = [x for x in text_splited if x.strip()]
texto_refNota3

In [ ]:
texto_valores = texto_refNota3

# Exemplo de uso
marcador1 = 'VALOR SERVIÇOS:'  # SAO PEDRO DA ALDEIA
marcador2 = 'LOCAL INCIDÊNCIA'
lista_val = encontrar_valores2(texto_valores, marcador1, marcador2)
lista_val

print(f'\nlista_val: {lista_val}  -->  {len(lista_val)}\n')

In [ ]:
# Nota 4 - SAO PEDRO DA ALDEIA
file_path = "pipeline_extracao_documentos/0_arquivos_teste_pipeline/pdf_raster/Sao_predo_aldeia/2023143.pdf"
doc2convert = "2023143.pdf"

image_2work, image_path = convertResize(doc2convert, file_path, image_resized_path)
imagem_gray = image_2work.convert('L')
texto_doc = (pytesseract.image_to_string(imagem_gray, lang='por'))
text_splited = texto_doc.split('\n')
texto_refNota4 = [x for x in text_splited if x.strip()]
texto_refNota4

In [ ]:
texto_valores = texto_refNota4

# Exemplo de uso
marcador1 = 'VALOR SERVIÇOS:'  # SAO PEDRO DA ALDEIA
marcador2 = 'LOCAL INCIDÊNCIA'
lista_val = encontrar_valores2(texto_valores, marcador1, marcador2)
lista_val

print(f'\nlista_val: {lista_val}  -->  {len(lista_val)}\n')

In [ ]:
# Nota 5 - Mesquita
file_path = "pipeline_extracao_documentos/0_arquivos_teste_pipeline/pdf_pesquisavel/Mesquita/2023__5.pdf"
doc2convert = "2023__5.pdf"

image_2work, image_path = convertResize(doc2convert, file_path, image_resized_path)
imagem_gray = image_2work.convert('L')
texto_doc = (pytesseract.image_to_string(imagem_gray, lang='por'))
text_splited = texto_doc.split('\n')
texto_refNota5 = [x for x in text_splited if x.strip()]
texto_refNota5

In [ ]:
texto_valores = texto_refNota5

# Exemplo de uso
marcador1 = 'VALOR SERVIÇOS:'  # SAO PEDRO DA ALDEIA
marcador2 = 'SIMPLES NACIONAL'
lista_val = encontrar_valores2(texto_valores, marcador1, marcador2)
lista_val

print(f'\nlista_val: {lista_val}  -->  {len(lista_val)}\n')

In [ ]:
texto_valores = texto_refNota5

# Exemplo de uso
marcador1 = 'DESC. COND:'  # SAO PEDRO DA ALDEIA
marcador2 = 'LOCAL INCIDÊNCIA'
lista_val = encontrar_valores2(texto_valores, marcador1, marcador2)
lista_val

print(f'\nlista_val: {lista_val}  -->  {len(lista_val)}\n')

In [ ]:
# 5. Valor total da nota
marcador_inicio = 'VALOR TOTAL DA NOTA:'
marcador_fim = 'CNAE -'
texto_valores = texto_refNota5

resultado = extrair_texto_entre_marcadores(texto_valores, marcador1, marcador2)
valor_total = resultado.replace(marcador_inicio, "").strip()
valor_total


In [ ]:
file_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento/Batch_20/fwdnotasfaltantesnosistemadeemissoim20734_106187/Doria Marinho 0295 Carlos Leandro.pdf"
original_file_name = os.path.basename(file_path)

section = "9. OUTRAS INFORMAÇOES / CRITICAS"
tipo = "field_box"
father_value = "5_frame_inf_criticas"
model_map = "MAGE"
de_para_pm = "PM_MAGE"

In [ ]:
data_box_valores = {}
data_box_valores['secao'] = section

#print(f'father_value: {father_value}, section: {section}, tipo: {tipo}, model_map: {model_map}, de_para_pm: {de_para_pm}, original_file_name: {original_file_name}\n')
# trato a imagem logo no começo
imagem_gray, image_resized_name = convert_resize_gray(original_file_name, file_path, image_resized_path)

# Estabeleco o filtro
filtered_boxes_info = frames_nf_v4_df[((frames_nf_v4_df['model'] == model_map) & (frames_nf_v4_df['type'] == tipo) & (frames_nf_v4_df['section_json'] == section) & (frames_nf_v4_df['father'] == father_value))]

for idx_frame, row_frame in filtered_boxes_info.iterrows():
    extracted_text_box = None
    reference = row_frame['reference']
    x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
    index = idx_frame
    
    label = row_frame['label']
    image_gray_croped = imagem_gray.crop((x0, y0, x1, y1))
    extracted_text_box = (pytesseract.image_to_string(image_gray_croped, lang='por'))
    #print(extracted_text_box)
    texto = extracted_text_box
    if label == "exigibilidade_iss":
        texto = extracted_text_box
        linhas = [linha for linha in texto.split('\n') if linha.strip()]
        marcador = "EXIGIBILIDADE ISS"
        value  = encontrar_valor_por_marcador(marcador, linhas)
        data_box_valores[label] = value
        
    if label == "regime_tributacao":
        texto = extracted_text_box
        linhas = [linha for linha in texto.split('\n') if linha.strip()]
        marcador = "REGIME TRIBUTAÇÃO"
        value = encontrar_valor_por_marcador(marcador, linhas)
        data_box_valores[label] = value
        
    if label == "simples_nacional":
        texto = extracted_text_box
        linhas = [linha for linha in texto.split('\n') if linha.strip()]
        marcador = "SIMPLES NACIONAL"
        value = encontrar_valor_por_marcador(marcador, linhas) 
        data_box_valores[label] = value 
    
    if label == "issqn_retido":
        texto = extracted_text_box 
        linhas = [linha for linha in texto.split('\n') if linha.strip()]
        marcador = "ISSQN RETIDO"
        valor = encontrar_valor_por_marcador(marcador, linhas)
        data_box_valores[label] = value 
        
        
    if label == "local_pretacao_servico":
        texto = extracted_text_box
        texto = texto_local_pretacao_servico.replace('\n\n', " ").strip()
        linhas = [linha for linha in texto.split('\n') if linha.strip()]
        marcador = "LOCAL. PRESTAÇÃO SERVIÇO"
        value = encontrar_valor_por_marcador(marcador, linhas)
        data_box_valores[label] = value 
        
    if label == "local_incidencia":
        texto = extracted_text_box
        linhas = [linha for linha in texto.split('\n') if linha.strip()] 
        marcador = "LOCAL INCIDÊNCIA"
        value = encontrar_valor_por_marcador(marcador, linhas)  
        data_box_valores[label] = value              



  

<mark> EXIGIBILIDADE ISS </mark>

In [ ]:
texto = texto_exigibilidade_iss
linhas = [linha for linha in texto.split('\n') if linha.strip()]
linhas

In [ ]:
marcador = "EXIGIBILIDADE ISS"
valor = encontrar_valor_por_marcador(marcador, linhas)
valor

<mark> REGIME TRIBUTAÇÃO </mark>

In [ ]:
texto = texto_regime_tributacao
linhas = [linha for linha in texto.split('\n') if linha.strip()]
linhas

In [ ]:
marcador = "REGIME TRIBUTAÇÃO"
valor = encontrar_valor_por_marcador(marcador, linhas)
valor

<mark> SIMPLES NACIONAL </mark>

In [ ]:
texto = texto_simples_nacional
linhas = [linha for linha in texto.split('\n') if linha.strip()]

In [ ]:
marcador = "SIMPLES NACIONAL"
valor = encontrar_valor_por_marcador(marcador, linhas)
valor

<mark> ISSQN RETIDO </mark>

In [ ]:
texto = texto_issqn_retido
linhas = [linha for linha in texto.split('\n') if linha.strip()]
linhas

In [ ]:
marcador = "ISSQN RETIDO"
valor = encontrar_valor_por_marcador(marcador, linhas)
valor

<mark> LOCAL. PRESTAÇÃO SERVIÇO </mark>

In [ ]:
texto = texto_local_pretacao_servico.replace('\n\n', " ").strip()
linhas = [linha for linha in texto.split('\n') if linha.strip()]
linhas

In [ ]:
marcador = "LOCAL. PRESTAÇÃO SERVIÇO"
valor = encontrar_valor_por_marcador(marcador, linhas)
valor

<mark> LOCAL INCIDÊNCIA </mark>

In [ ]:
texto = texto_local_incidencia
linhas = [linha for linha in texto.split('\n') if linha.strip()]
linhas

In [ ]:
marcador = "LOCAL INCIDÊNCIA"
valor = encontrar_valor_por_marcador(marcador, linhas)
valor

In [ ]:
def split_string_with_exceptions(s, phrases):
    safe_phrases = {phrase: phrase.replace(' ', '_') for phrase in phrases}

    for phrase, safe_phrase in safe_phrases.items():
        s = s.replace(phrase, safe_phrase)

    words = s.split()

    for safe_phrase, phrase in safe_phrases.items():
        words = [word.replace(safe_phrase, phrase) for word in words]

    return words

In [ ]:
def get_original_phrase(safe_word, safe_phrases_to_original):
    return safe_phrases_to_original.get(safe_word, safe_word)

In [ ]:
# Usando a função
safe_phrases_to_original = {"Microempresa_municipal": "Microempresa municipal", "Magé_-_RJ": "RJ Magé"}

In [ ]:
original_phrase = get_original_phrase("Microempresa_municipal", safe_phrases_to_original)
print(original_phrase)  # Deve imprimir: "Sociedade Limitada"

In [ ]:
def remove_word_from_list(word_list, word_to_remove):
    return [word for word in word_list if word != word_to_remove]


In [ ]:
# Usando a função
word_list = ["VALOR_IR:", "RJ_Magé", "DESC._COND:"]
word_to_remove = "DESC._COND:"

In [ ]:
new_word_list = remove_word_from_list(impostos, word_to_remove)
print(new_word_list)  # Deve imprimir: ["RJ_Magé", "outra_palavra"]

In [ ]:
    secao = "6. CNAE e Item da Lista de Serviços"
    try:
        nd_data_CNAE = {}
        nd_data_CNAE['secao'] = secao
        f_frame_name = "4_frame_cnae_itens_servico"   
        Texto_extraido = executa_model_frame(model, secao, f_frame_name)
        text_splited = Texto_extraido.split('\n')
        # Processando CNAE
        cnae_line = [line for line in text_splited if 'CNAE' in line][0]
        cnae_number = int(extract_number(cnae_line))
        cnae_value = cnae_dict.get(cnae_number, "Valor não encontrado")
        if cnae_value == 'Valor não encontrado':
            cnae_value = processa_cnae_outros(cnae_line)
            cnae_value = cnae_value.upper()
            nd_data_CNAE['cnae'] = cnae_value
        else:
            cnae_value = cnae_value.upper()
            cnae_value = str(cnae_number) + " - " + cnae_value
            nd_data_CNAE['cnae'] = cnae_value
            nd_data_CNAE['item_lista_servicos'] = item_servico_value
    except Exception as e:
        print(f"Erro busca cnae: {e}")    

    try:
        item_servico_line = [line for line in text_splited if 'Item da Lista de Serviços' in line][0]
        item_servico_number = float(extract_number(item_servico_line))
        item_servico_value = item_servico_dict.get(item_servico_number, "Valor não encontrado")
        item_servico_value = item_servico_value.upper()
        item_servico_value = str(item_servico_number) + " - " + item_servico_value
        nd_data_CNAE['item_lista_servicos'] = item_servico_value
    except Exception as e:
        print(f"Erro busca Itens de servico: {e}")  

In [ ]:
    # templates = {
    #     ("PM_MAGE", "30.693.231/0001-99"): "MAGE_MAICON",
    #     ("PM_MAGE", "23.317.112/0001-76"): "MAGE_MFF",
    #     ("PM_MAGE", "06.083.610/0001-82"): "MAGE_DORIA",      
    #     ("PM_MAGE", None): "MAGE",
    #     ("PREFEITURA MUNICIPAL DE SAO PEDRO DA ALDEIA", "47.945.459/0001-21"): "SAO_PEDRO_GOAT",
    #     ("PREFEITURA MUNICIPAL DE SAO PEDRO DA ALDEIA", "68.687.722/0001-08"): "SAO_PEDRO_GM",
    #     ("PREFEITURA MUNICIPAL DE SAO PEDRO DA ALDEIA", "34.230.979/0038-06"): "SAO_PEDRO_SUPERMIX",
    #     ("PREFEITURA MUNICIPAL DE SAO PEDRO DA ALDEIA", None): "SAO_PEDRO",
    #     ("Pague agora com o seu Pix", None): "NAO_PROCESSAR",
    #     # ... adicione outras combinações aqui
    # }

In [ ]:

    imagem_gray, image_resized_name = convert_resize_gray(original_file_name, file_path, image_resized_path)
    
    for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
        label_value = row_frame['label']
        reference_value = row_frame['reference']
        f_type = row_frame['type']
        frame_id = row_frame['id']
        if row_frame['label']  == "1_frame_prefeitura_nf":
            x0, y0, x1, y1 = (row_frame['x0_p'], row_frame['y0_p'], row_frame['x1_p'], row_frame['y1_p']) if pdf_pesquisavel_map else (row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1'])
            texto_extraido = extract_text_PIL(imagem_gray, (x0, y0, x1, y1))
            #print(texto_extraido)
            label_value = row_frame['label']
            #print("label_value", label_value)
            values = texto_extraido.split('\n')
            for index_sframe, row_sframe in filtered_sframe_field_nf_v4_df.iterrows():
                label_value = row_sframe['label']
                #print("label_value", label_value)
                if label_value == "nome_prefeitura":
                    reference_value = row_sframe['reference']
                    for value in values:
                        result = process_line(value, reference_value, label_value)
                        if result:
                            data_extrated_prefeitura.update(result)
                elif label_value == "secretaria":
                    reference_value = row_sframe['reference']
                    for value in values:
                        result = process_line(value, reference_value, label_value)
                        if result:
                            data_extrated_prefeitura.update(result) 
                elif label_value == "tipo_nota_fiscal":
                    reference_value = row_sframe['reference']  
                    for value in values:
                        result = process_line(value, reference_value, label_value)
                        if result:
                            data_extrated_prefeitura.update(result)
        
        if row_frame['label']  == "1_frame_dados_nf":
            x0, y0, x1, y1 = (row_frame['x0_p'], row_frame['y0_p'], row_frame['x1_p'], row_frame['y1_p']) if pdf_pesquisavel else (row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1'])
            texto_extraido = extract_text_PIL(imagem_gray, (x0, y0, x1, y1))
            #print(texto_extraido)
            text_splited = texto_extraido_cabecalho(texto_extraido)
            keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']

            string_pesquisa = "Número da Nota:"
            texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)         
            data_extrated_prefeitura['numero_nota_fiscal'] = texto

            string_pesquisa = "Competência:"
            texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            data_extrated_prefeitura['competencia'] = texto

            string_pesquisa = "ata e Hora da Emissão:"
            texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            data_extrated_prefeitura['dt_hr_emissao'] = texto

            string_pesquisa = "Código Verificação:"
            texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            data_extrated_prefeitura['codigo_verificacao'] = texto
            #print(f'{frame_id:>5}  {f_type:>20} | {label_value:>30} |  ref:{reference_value:>30}  | x0: {x0:>6} y0: {y0:>6} x1: {x1:>6} y1: {y1:>6}')

In [ ]:
# 5. XXX Ajusta texto
def texto_extraido(texto):
    #0. Tratamento da string
    text_splited = texto.split('\n')
    text_splited = [s.replace(":", "") for s in text_splited]
    text_splited = [x for x in text_splited if x.strip()]
    text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao
    return text_splited


# 6. XXX Ajusta texto para PDF RASTER NO CABECALHO
def texto_extraido_cabecalho(texto):
    #0. Tratamento da string
    text_splited = texto.split('\n')
    text_splited = [x for x in text_splited if x.strip()]
    text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao
    text_splited = [s.replace(")", "").strip() for s in text_splited] #depende da situaçao
    return text_splited


# 6. XXX Ajusta texto para PDF_Pesquisavel-  NO CABECALHO
def texto_extraido_nf(texto):
    #0. Tratamento da string
    text_splited = texto.split('\n')
    text_splited = [x for x in text_splited if x.strip()]
    text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao
    return text_splited

In [ ]:
def extract_text_from_coordinates(image, coordinates, config):
    x0, y0, x1, y1 = coordinates
    frame_image = image.crop((x0, y0, x1, y1))
    print(x0, y0, x1, y1)
    frame_image
    extracted_text = pytesseract.image_to_string(frame_image, lang='por', config=config).strip()
    return extracted_text

def extract_text_from_coordinates_2(image, coordinates, config):
    x0, y0, x1, y1 = coordinates
    print(x0, y0, x1, y1)
    frame_image = image.crop((x0, y0, x1, y1))
    extracted_text = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config_1).strip()
    return extracted_text


In [ ]:
def extract_dados_comple_obs(modelo, frame_father, section):
    
    data_dados_complementares = {}
    #frame_label = frame_father
    
    # 1. Filtrando o frames_info para buscar os dados de corte
    filtered_frames_info = frames_info[(frames_info['label'] == frame_father) & (frames_info['model'] == modelo)]

    # 2. Filtrando o sframe_fields_info para buscar os dados dos campos que estao nos frames
    filtered_sframe_fields_info = sframe_fields_info[(sframe_fields_info['father'] == frame_father) & (sframe_fields_info['model'] == modelo)]

    for index_frame, row_frame in filtered_frames_info.iterrows():
        
        x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
        extracted_text_box = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
        
        print("{:<5} {:<10} {:<30} {:<20} {:<20} {:<7} {:<7} {:<7} {:<7}".format(row_frame['seq'], row_frame['model'], row_frame['father'], row_frame['label'], row_frame['reference'], row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1'] ))
        for index_field, row_field in filtered_sframe_fields_info.iterrows():
            #print("{:<5} {:<10} {:<30} {:<20} {:<20}".format(row_field['seq'], row_field['model'], row_field['father'], row_field['label'], row_field['reference']))
            
            if frame_father == "5_frame_dados_complementares":
                nf_data_dados_complementares = {}
                nf_data_dados_complementares['section'] = section
                
                # Remove a primeira ocorrência de "Observação:"
                text = re.sub(r'^DADOS COMPLEMENTARES', '', extracted_text_box, count=1)
                if text == '':
                    text = None
                    nf_data_dados_complementares['dados_complementares'] = text
                else:    
                    # Extrair texto dentro do retângulo
                    nf_data_dados_complementares['dados_complementares'] = text.strip()
                    
                return nf_data_dados_complementares                
                
            elif frame_father == "5_frame_observacao":
                nf_data_observacao = {}
                nf_data_observacao['section'] = section 
                                # Remove a primeira ocorrência de "Observação:"
                text = re.sub(r'^Observação:', '', extracted_text_box, count=1)

                # Remover quebras de linha
                text = text.replace('\n', ' ')

                # Extrair texto dentro do retângulo
                nf_data_observacao['observacao'] = text.strip()
                
                return nf_data_observacao 

In [ ]:
def extract_fields_tomador_cnpj(text):
    nf_data_tomador_cnpj = {}
    
    # Extrair CPF/CNPJ com máscara 1
    if "CPF/CNPJ:" in text:
        cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', text)
        if cpf_cnpj_formatado_match:
                        nf_data_tomador_cnpj['cpf_cnpj_com_mascara'] = cpf_cnpj_formatado_match.group(1)
                        nf_data_tomador_cnpj['cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))

    
    # Extrair Telefone
    telefone_match = re.search(r'Telefone:\s+(.+)', text)
    if telefone_match:
        telefone_str = telefone_match.group(1)
        if telefone_str == 'Inscrição Estadual:':
            nf_data_tomador_cnpj['telefone'] = None  # Valor padrão quando não há correspondência
        elif telefone_str == '':
            nf_data_tomador_cnpj['telefone'] = None  # Valor padrão quando não há correspondência
                    
        else:    
            nf_data_tomador_cnpj['telefone'] = telefone_match.group(1)
            
    
    # Extrair Inscrição Municipal
    inscricao_municipal_match = re.search(r'Inscrição Municipal:\s+(.+)', text)
    if inscricao_municipal_match:
        inscricao_municipal_str = inscricao_municipal_match.group(1)
        if inscricao_municipal_str == "Telefone:": 
            nf_data_tomador_cnpj['inscricao_municipal'] = None
        else:    
            nf_data_tomador_cnpj['inscricao_municipal'] = inscricao_municipal_str
    
    insc_municipal_match = re.search(r'INSC:MUNICIPAL:\s+(.+)', text)
    if insc_municipal_match:
        insc_municipal_str = insc_municipal_match.group(1)
        if insc_municipal_str == "Telefone:":
            nf_data_tomador_cnpj['inscricao_municipal'] = None
        else:    
            nf_data_tomador_cnpj['inscricao_municipal'] = insc_municipal_str
    else:
        nf_data_tomador_cnpj['inscricao_municipal'] = None
            
    
    return nf_data_tomador_cnpj 


In [ ]:
# Funcao importante - process_line
def process_line(value, reference, label):
    name_match = re.search(fr'{reference} (.+)', value)
    if name_match:
        extracted_value = reference + " " + name_match.group(1)
        return {label: extracted_value}
    return None


def extract_fb_outras_inf(modelo, father_value, section):

    data_box_valores = {}
    data_box_valores['secao'] = section
    filtered_boxes_info = field_boxes_info[(field_boxes_info['father'] == father_value) & (field_boxes_info['model'] == model)]
    # Iterate nas informações dos boxes de fields e extraia o texto de cada field
    for index_field, row_field in filtered_boxes_info.iterrows():
        
        string_pesquisa = row_field['reference']
        x0, y0, x1, y1 = row_field['x0'], row_field['y0'], row_field['x1'], row_field['y1']
        extracted_text_box = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
        label = row_field['label']
        #print(f'extracted_text_box {extracted_text_box}, label {label}')
        text = extracted_text_box.replace('\n', '')
        if text.startswith(string_pesquisa):
            #print("aqui:", text)
            text = text[len(label):].strip()
            data_box_valores[label] = text
    
    return   data_box_valores  


def extract_text_from_frame(image, coordinates, config):
    x0, y0, x1, y1 = coordinates
    frame_image = image.crop((x0, y0, x1, y1))
    extracted_text = pytesseract.image_to_string(frame_image, lang='por', config=config).strip()
    return extracted_text 


# 2. Efetua OCR no documento (area parao do texto da NF)
def ocr_RasterPDF_free(image_name, vx0, vy0, vx1, vy1):
    
    analise_pesquisa_nf = {}
    # 1. Definindo as coordenadas do frame
    x0 = vx0
    y0 = vy0
    x1= vx1
    y1 = vy1

    # 2. Definir frame_image
    frame_image = image_name.crop((x0, y0, x1, y1))

    # 3. Extraia texto usando OCR com configuração de idioma padrão para este frame
    extracted_text_frame = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config).strip()

    # 4. Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
    values = extracted_text_frame.split('\n')
    return values, extracted_text_frame 


# 3. Efetua OCR no documento (area parao do texto da NF)
def ocr_RasterPDF(image_name):
    
    analise_pesquisa_nf = {}
    # 1. Definindo as coordenadas do frame
    x0 = 406
    y0 = 0
    x1= 1540
    y1 = 380

    # 2. Definir frame_image
    frame_image = image_name.crop((x0, y0, x1, y1))

    # 3. Extraia texto usando OCR com configuração de idioma padrão para este frame
    extracted_text_frame = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config).strip()

    # 4. Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
    values = extracted_text_frame.split('\n')
    return values, extracted_text_frame 

In [ ]:
# 2. XXX Baseado no texto extraido, cria um dicionario com os dados extraidos
def cria_guarda_doc_ref_R_PDF(idx, row, de_para_pm, model_map, original_file_name, file_path, image_resized_path):
    
    guarda_texto_doc = {}
    
    texto_documento = extracao_texto_ref(idx, row, model_map, original_file_name, file_path, image_resized_path)
    
    guarda_texto_doc['document_unique_id'] = idx
    guarda_texto_doc['original_file_name'] = original_file_name
    guarda_texto_doc['texto_documento'] = texto_documento
    
    return guarda_texto_doc, texto_documento

In [ ]:
def extrai_geral_data_R_PDF(idx, row, de_para_pm, model_map, marcador_inicio, marcador_fim, def_replace, original_file_name, file_path):
    
    nf_data_geral = {}
    x0 = 1
    y0 = 1
    x1 = 2067
    y1 = 2923
    image_2work, image_resized_name = convertResize(original_file_name, file_path, image_resized_path)
    texto_extraido = extract_text_PIL(image_2work, (x0, y0, x1, y1))
    text_splited = texto_extraido.split('\n')
    
    marcador_inicio = marcador_inicio
    marcador_fim = marcador_fim
    resultado = extrair_texto_entre_marcadores(text_splited, marcador_inicio, marcador_fim)
    if def_replace:
        resultado = resultado.replace(marcador_inicio, "").strip()
        return resultado
    else:
        return resultado    

In [ ]:
file_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento/Batch_20/fwdnotasfaltantesnosistemadeemissoim20734_106187/Doria Marinho 0301 Ultrascan.pdf"
def_replace = True

#section = "4. DESCRIMINACAO DOS SERVIÇOS"
marcador_inicio = 'DISCRIMINAÇÃO DOS SERVIÇOS'
marcador_fim = 'VALOR TOTAL DA NOTA:'

original_file_name = "Doria Marinho 0301 Ultrascan.pdf"



In [ ]:
model_map = 'MAGE_DORIA'
section = "4. DESCRIMINACAO DOS SERVIÇOS"
label = "discriminacao_servicos"
tipo = "field_box"
original_file_name = "Doria Marinho 0301 Ultrascan.pdf"
de_para_pm = "PM_MAGE"

file_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento/Batch_20/fwdnotasfaltantesnosistemadeemissoim20734_106187/Doria Marinho 0296 Vanisa (Cancelada).pdf"

In [ ]:
  def extrai_servicos_R_PDF(idx, row, pdf_pesquisavel_map, de_para_pm, model_map, f_0, f_1, original_file_name, file_path):
    
    section = "4. DESCRIMINACAO DOS SERVIÇOS"
    tipo = "field_box"
    label = "discriminacao_servicos"
    nf_data_servico = {}
    pdf_pesquisavel_map = False
    
 
    image_2work, image_resized_name = convertResize(original_file_name, file_path, image_resized_path)
    
    coordinates = get_coordinates_filter_R_PDF(model_map, tipo, label, section)
    x0, y0, x1, y1 = coordinates[0]
    #print(f'1. : (coordenadas originais):  x0:{x0} y0:{y0} x1:{x1} y1:{y1}\n')
    y0 = y0 * f_0
    y1 = y1 * f_1
    print(f'coordenadas:   x0:{x0} y0:{y0} x1:{x1} y1:{y1}: f_0:{f_0}, f_1: {f_1}\n')
    texto_extraido = extract_text_PIL(image_2work, (x0, y0, x1, y1))
    #text = texto_extraido.replace('\n', ' ')
    
    print(f'texto_extraido:\n{texto_extraido}\n')
    label = "DISCRIMINAÇÃO DOS SERVIÇOS"
    # if text.startswith(label):
    #   text = text[len(label):].strip()
    #   nf_data_servico['discriminacao_servicos'] = text        
      
          
    return nf_data_servico   

In [ ]:
# Descriçao
marcador_inicio = 'DISCRIMINAÇÃO DOS SERVIÇOS'
marcador_fim = 'VALOR TOTAL DA NOTA:'

resultado = extrair_texto_entre_marcadores(text_splited, marcador_inicio, marcador_fim)
resultado = resultado.replace(marcador_inicio, "").strip()
resultado

In [ ]:
# Outras Informacoes criticas
marcador_inicio = 'EXIGIBILIDADE ISS REGIME TRIBUTAÇÃO SIMPLES NACIONAL ISSQN RETIDO LOCAL. PRESTAÇÃO LOCAL INCIDÊNCIA'
marcador_fim = 'Observação:'

resultado = extrair_texto_entre_marcadores(text_splited, marcador_inicio, marcador_fim)
resultado
resultado = resultado.replace(marcador_inicio, "").strip()
resultado

In [ ]:
# Outras Informacoes criticas
marcador_inicio = 'EXIGIBILIDADE ISS REGIME TRIBUTAÇÃO SIMPLES NACIONAL ISSQN RETIDO LOCAL. PRESTAÇÃO LOCAL INCIDÊNCIA'
marcador_fim = 'Observação:'

resultado = extrair_texto_entre_marcadores(text_splited, marcador_inicio, marcador_fim)
resultado
resultado = resultado.replace(marcador_inicio, "").strip()
resultado

In [ ]:
# valores dos impostos
marcador_inicio = 'i Fi"'
marcador_fim = 'DADOS COMPLEMENTARES'

resultado = extrair_texto_entre_marcadores(text_splited, marcador_inicio, marcador_fim)
resultado
resultado = resultado.replace(marcador_inicio, "").strip()
resultado

In [ ]:
resultado = resultado.replace(marcador_inicio, "").strip()
resultado

In [ ]:
# ITEM DA LISTA DE SERVICO
marcador_inicio = 'Item da Lista de Serviços -'
marcador_fim = 'VALOR SERVIÇOS:'

resultado = extrair_texto_entre_marcadores(text_splited, marcador_inicio, marcador_fim)
resultado = resultado.replace(marcador_inicio, "").strip()
resultado

In [ ]:
# CNAE
marcador_inicio = 'CNAE - '
marcador_fim = 'Item da Lista de Serviços -'

resultado = extrair_texto_entre_marcadores(text_splited, marcador_inicio, marcador_fim)
resultado = resultado.replace(marcador_inicio, "").strip()
resultado

In [ ]:
# Valor total da nota
marcador_inicio = 'VALOR TOTAL DA NOTA:'
marcador_fim = 'CNAE -'

resultado = extrair_texto_entre_marcadores(text_splited, marcador_inicio, marcador_fim)
resultado = resultado.replace(marcador_inicio, "").strip()
resultado

In [ ]:
# Descriçao
marcador_inicio = 'DISCRIMINAÇÃO DOS SERVIÇOS'
marcador_fim = 'VALOR TOTAL DA NOTA:'

resultado = extrair_texto_entre_marcadores(text_splited, marcador_inicio, marcador_fim)
resultado = resultado.replace(marcador_inicio, "").strip()
resultado

In [ ]:
# Razao Social
marcador_inicio = 'Nome/Razão Social:'
marcador_fim = 'Nome de Fantasia:'

resultado = extrair_texto_entre_marcadores(text_splited, marcador_inicio, marcador_fim)
resultado = resultado.replace(marcador_inicio, "").strip()
resultado

In [ ]:
resultado = resultado.replace(marcador_inicio, "").strip()
resultado

In [ ]:
# Codigo de verificaçao
marcador_inicio = 'Código Verificação'
marcador_fim = 'PRESTADOR DE SERVIÇOS'

resultado = extrair_texto_entre_marcadores(text_splited, marcador_inicio, marcador_fim)
resultado = resultado.replace(marcador_inicio, "").strip()
resultado

In [ ]:
text_splited = [x for x in text_splited if x.strip()]

In [ ]:
text_splited

In [ ]:
def extrair_texto_entre_marcadores(texto, marcador_inicio, marcador_fim):
    try:
        # Encontra os índices dos marcadores de início e fim
        indice_inicio = next(i for i, s in enumerate(texto) if s.startswith(marcador_inicio)) + 1
        indice_fim = next(i for i, s in enumerate(texto) if s.startswith(marcador_fim))
        
        # Extrai e retorna o texto entre os marcadores
        return " ".join(texto[indice_inicio:indice_fim])
    except StopIteration:
        # Retorna None se algum dos marcadores não for encontrado
        return None

In [ ]:
if indice_inicio == indice_fim:
    inicio = texto[indice_inicio].find(marcador_inicio) + len(marcador_inicio)
    fim = texto[indice_fim].find(marcador_fim)
    resultado = texto[indice_inicio][inicio:fim].strip()
    resultado = resultado.replace(marcador_inicio, "").strip()  # Remove o marcador de início
    return resultado


In [ ]:
marcador_inicio = 'TOMADOR DE SERVIÇOS'
marcador_fim = 'Telefone'

resultado = extrair_texto_entre_marcadores(text_splited, marcador_inicio, marcador_fim)
resultado = resultado.replace(marcador_inicio, "").strip()
resultado


In [ ]:
marcador_inicio = 'VALOR TOTAL DA NOTA'
marcador_fim = 'CNAE'

In [ ]:
marcador_inicio = 'DISCRIMINAÇÃO DOS SERVIÇOS'
marcador_fim = 'VALOR TOTAL DA NOTA'

In [ ]:
resultado = extrair_texto_entre_marcadores(text_splited, marcador_inicio, marcador_fim)
resultado

In [ ]:
marcador_inicio

In [ ]:
resultado = resultado.replace(marcador_inicio, "").strip()
resultado

In [ ]:
def extrai_prestador_R_PDF_RESERVA(idx, row, pdf_pesquisavel_map, de_para_pm, model_map, f_0, f_1, original_file_name, file_path):

    nf_data_prestador = {}
    dic_erros = {}
    message_erro = []
    section = "2. PRESTADOR DE SERVIÇO"
    pdf_pesquisavel_map = False
    nf_data_prestador['secao'] = "2. PRESTADOR DE SERVIÇO"

    process = ['2_frame_cnpj_prestador', '2_frame_inscricao_prestador', '2_frame_dados_prestador']
    tipo = "frame"

    image_2work, image_resized_name = convertResize(original_file_name, file_path, image_resized_path)
    
    for label in process:
        seq = process.index(label) + 1
        if label == "2_frame_cnpj_prestador":
            coordinates = get_coordinates_filter_R_PDF(model_map, tipo, label, section)
            x0, y0, x1, y1 = coordinates[0]
            texto_extraido = extract_text_PIL(image_2work, (x0, y0, x1, y1))
            #print(f'texto_extraido: {texto_extraido}\n\n')
            text_splited = texto_extraido_nf(texto_extraido)
            
            keyword_list = ['CPF/CNPJ:', 'Telefone:']
            string_pesquisa = "CPF/CNPJ:"
            texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', texto)
            if cpf_cnpj_formatado_match:
                nf_data_prestador['p_cpf_cnpj_com_mascara'] = cpf_cnpj_formatado_match.group(1)
                nf_data_prestador['p_cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))
            
            try:
                string_pesquisa = "Telefone:"  
                texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                print(texto) 
                telefone_match = re.search(r'Telefone:\s+([0-9.\s-]+)', texto) 
                if telefone_match: 
                    telefone_str = telefone_match.group(1)
                    # Remover quebras de linha
                    telefone_str = telefone_str.replace('.', '')
                    telefone_str = telefone_str.replace('\n', '')
                            
                    nf_data_prestador['p_telefone'] = telefone_str
                else:
                    nf_data_prestador['p_telefone'] = None
            except Exception as e:
                msg = (f"doc: | {e}")
                message_erro.append(msg)
                nf_data_prestador['p_telefone'] = None   
                
        if label == "2_frame_inscricao_prestador":
            coordinates = get_coordinates_filter_R_PDF(model_map, tipo, label, section)
            x0, y0, x1, y1 = coordinates[0]
            texto_extraido = extract_text_PIL(image_2work, (x0, y0, x1, y1))
            
            print(f'texto_extraido: {texto_extraido} | {original_file_name} \n\n')
            try:
                text_splited = texto_extraido.split('\n')
                text_splited = texto_extraido(texto_extraido)
                text_splited = [x for x in text_splited if x.strip()]
                keyword_list = ['Inscrição Municipal:', 'Inscrição Estadual:']
                string_pesquisa = "Inscrição Municipal:"
                texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                nf_data_prestador['p_inscricao_municipal'] = texto
                
                string_pesquisa = "Inscrição Estadual:"
                texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                nf_data_prestador['p_inscricao_estadual'] = texto
            
            except Exception as e:
                new_row = {
                    "row_index": idx,  # Substitua 'index' pela variável que contém o índice da linha atual
                    "erro_inscricao": str(e),
                    "file": original_file_name,
                    "process_label": label
                }   
                dic_erros.update(new_row)
                nf_data_prestador['p_inscricao_municipal'] = None
                nf_data_prestador['p_inscricao_estadual'] = None
            



    return nf_data_prestador, dic_erros 

In [ ]:
marcador_inicio = marc_ini[0]
marcador_fim = marc_fim[0]
resultado = extrair_texto_entre_marcadores(texto_documento_uso, marcador_inicio, marcador_fim)
discriminacao_servico = resultado.replace(marcador_inicio, "").strip()
discriminacao_servico

In [ ]:
def extrai_prestador_R_PDF(row, pdf_pesquisavel_map, de_para_pm, model_map, f_0, f_1, original_file_name, file_path):

    nf_data_prestador = {}
    message_erro = []
    section = "2. PRESTADOR DE SERVIÇO"
    pdf_pesquisavel_map = False
    nf_data_prestador['secao'] = "2. PRESTADOR DE SERVIÇO"

    process = ['2_frame_cnpj_prestador', '2_frame_inscricao_prestador', '2_frame_dados_prestador']
    tipo = "frame"

    image_2work, image_resized_name = convertResize(original_file_name, file_path, image_resized_path)
    
    for label in process:
        seq = process.index(label) + 1
        if label == "2_frame_cnpj_prestador":
            coordinates = get_coordinates_filter_R_PDF(model_map, tipo, label, section)
            x0, y0, x1, y1 = coordinates[0]
            texto_extraido = extract_text_PIL(image_2work, (x0, y0, x1, y1))
            #print(f'texto_extraido: {texto_extraido}\n\n')
            text_splited = texto_extraido_nf(texto_extraido)
            
            keyword_list = ['CPF/CNPJ:', 'Telefone:']
            string_pesquisa = "CPF/CNPJ:"
            texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', texto)
            if cpf_cnpj_formatado_match:
                nf_data_prestador['p_cpf_cnpj_com_mascara'] = cpf_cnpj_formatado_match.group(1)
                nf_data_prestador['p_cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))
            
            try:
                string_pesquisa = "Telefone:"  
                texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                print(texto) 
                telefone_match = re.search(r'Telefone:\s+([0-9.\s-]+)', texto) 
                if telefone_match: 
                    telefone_str = telefone_match.group(1)
                    # Remover quebras de linha
                    telefone_str = telefone_str.replace('.', '')
                    telefone_str = telefone_str.replace('\n', '')
                            
                    nf_data_prestador['p_telefone'] = telefone_str
                else:
                    nf_data_prestador['p_telefone'] = None
            except Exception as e:
                msg = (f"doc: | {e}")
                message_erro.append(msg)
                nf_data_prestador['p_telefone'] = None   
                
        if label == "2_frame_inscricao_prestador":
            coordinates = get_coordinates_filter_R_PDF(model_map, tipo, label, section)
            x0, y0, x1, y1 = coordinates[0]
            texto_extraido = extract_text_PIL(image_2work, (x0, y0, x1, y1))
            print(f'texto_extraido: {texto_extraido} | {original_file_name} \n\n')
            try:
                text_splited = texto_extraido(texto_extraido)
                keyword_list = ['Inscrição Municipal:', 'Inscrição Estadual:']
                string_pesquisa = "Inscrição Municipal:"
                texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                nf_data_prestador['p_inscricao_municipal'] = texto
                
                string_pesquisa = "Inscrição Estadual:"
                texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                nf_data_prestador['p_inscricao_estadual'] = texto
            
            except Exception as e:
                msg = (f"doc: {original_file_name} | {e}")
                nf_data_prestador['p_inscricao_municipal'] = None
                nf_data_prestador['p_inscricao_estadual'] = None
                message_erro.append(msg)
            
            
            
         
            


    return nf_data_prestador  

In [ ]:
 # Remover quebras de linha e rótulo
        text = text.replace('\n', ' ')
        if text.startswith(label):
            text = text[len(label):].strip()

In [ ]:
# 2. PRESTADOR DE SERVIÇO
def extrai_prestador_R_PDF(row, pdf_pesquisavel_map, de_para_pm, model_map, f_0, f_1, original_file_name, file_path):
    
    nf_data_prestador = {}
    section = "2. PRESTADOR DE SERVIÇO"
    pdf_pesquisavel_map = False
    
    process = ['2_frame_cnpj_prestador', '2_frame_inscricao_prestador', '2_frame_dados_prestador']

    tipo = "frame"

    coordinates = get_coordinates_filter(pdf_pesquisavel_map=pdf_pesquisavel_map, model=model_map, tipo=tipo, label=label, section=section)
    x0, y0, x1, y1 = coordinates[0]
    #print(label)
    #print(x0,x1,y0,y1)
    y0 = y0 * f_0
    y1 = y1 * f_1
    text = page.get_text("text", clip=(x0, y0, x1, y1))
    nf_data_prestador = novaextra.extract_fields_prestador(text)

           

    pdf_document.close()

    return nf_data_prestador 

# 2.B PRESTADOR DE SERVIÇO - RASTER
def processar_prestador_raster_pdf(row, pdf_pesquisavel_map, model_map, original_file_name, file_path):
    
    section = "2. PRESTADOR DE SERVIÇO"
    pdf_pesquisavel_map = False
    
    tipo= "frame"
    message_erro = []
    nf_dados_prestador = {}
    model = row['model']
    print(model)

    process_prestador = ['2_frame_cnpj_prestador', '2_frame_inscricao_prestador', '2_frame_dados_prestador']
    image_2work, image_resized_name = convertResize(original_file_name, file_path, image_resized_path)
    for father in process_prestador:
            label = father
            if label == "2_frame_cnpj_prestador": 
                coordinates = get_coordinates_filter(model_map, tipo, label, section)
                x0, y0, x1, y1 = coordinates[0]
                extract_text = extract_text_PIL(image_2work, (x0, y0, x1, y1))
                #text_splited = texto_extraido_nf(extract_text)
                if "CPF/CNPJ:" in extract_text:
                    cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', extract_text)
                    if cpf_cnpj_formatado_match:
                        texto = cpf_cnpj_formatado_match.group(1)
                    
                        nf_dados_prestador['cpf_cnpj_com_mascara'] = texto
                        nf_dados_prestador['cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))
                else:
                    nf_dados_prestador['cpf_cnpj_com_mascara'] = 'None'
                    nf_dados_prestador['cpf_cnpj_sem_mascara'] = 'None'           
                        
                telefone_str = None
                #telefone_match = re.search(r'Telefone:\s+([0-9.\s-])', text)
                telefone_match = re.search(r'Telefone:\s+([0-9.\s-]+)', extract_text)
                if telefone_match: 
                    telefone_str = telefone_match.group(1)
                    # Remover quebras de linha
                    telefone_str = telefone_str.replace('.', '')
                    telefone_str = telefone_str.replace('\n', '')
                            
                    nf_dados_prestador['telefone'] = telefone_str
                else:
                    nf_dados_prestador['telefone'] = 'None'
                    
            # if father_to_process == "2_frame_cnpj_prestador":
            elif label == '2_frame_inscricao_prestador':
                coordinates = get_coordinates_filter(model_map, tipo, label, section)
                x0, y0, x1, y1 = coordinates[0]
                extract_text = extract_text_PIL(image_2work, (x0, y0, x1, y1))
                
                text_splited = texto_extraido_nf(extract_text)
                
                
                # Extrair Inscrição Estadual
                inscricao_estadual_match = re.search(r'Inscrição Estadual:\s+(.+)', extract_text)
                if inscricao_estadual_match:
                    inscricao_estadual_str = inscricao_estadual_match.group(1)
                    if inscricao_estadual_str == 'Nome/Razão Social:':
                        nf_inscr_estadual = 'None'
                        nf_dados_prestador['inscricao_estadual'] = "NONE"
                    else:
                        nf_inscr_estadual = inscricao_estadual_match.group(1)
                        nf_dados_prestador['inscricao_estadual'] = inscricao_estadual_match.group(1) 
                

            elif label == '2_frame_dados_prestador':
                dados_prestador = {}
                coordinates = get_coordinates_filter(model_map, tipo, label, section)
                x0, y0, x1, y1 = coordinates[0]
                
                extract_text = extract_text_PIL(image_2work, (x0, y0, x1, y1))
                dados_prestador = extract_fields_prestador(extract_text)
                
                text_splited = texto_extraido_nf(extract_text)
                keyword_list = ['Nome/Razão Social:', 'Nome de Fantasia:', 'Endereço:', 'E-mail:']
                string_pesquisa = "Nome/Razão Social:"
                texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            
                nf_dados_prestador['razao_social'] = texto

                string_pesquisa = "Nome de Fantasia:"
                texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                nf_dados_prestador['nome_fantasia'] = texto
                
                string_pesquisa = "Endereço:"
                texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                nf_dados_prestador['endereco'] = texto
                
                string_pesquisa = "E-mail:"
                texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                nf_dados_prestador['email'] = texto
                
    return nf_dados_prestador  

In [ ]:
def processar_prestador_raster_pdf(row, pdf_pesquisavel_map, model_map, original_file_name, file_path):
    
    section = "2. PRESTADOR DE SERVIÇO"
    pdf_pesquisavel_map = False
    
    tipo= "frame"
    message_erro = []
    nf_dados_prestador = {}
    model = row['model']
    print(model)

    process_prestador = ['2_frame_cnpj_prestador', '2_frame_inscricao_prestador', '2_frame_dados_prestador']
    image_2work, image_resized_name = convertResize(original_file_name, file_path, image_resized_path)
    for father in process_prestador:
            label = father
            if label == "2_frame_cnpj_prestador": 
                coordinates = get_coordinates_filter(model_map, tipo, label, section)
                x0, y0, x1, y1 = coordinates[0]
                extract_text = extract_text_PIL(image_2work, (x0, y0, x1, y1))
                #text_splited = texto_extraido_nf(extract_text)
                if "CPF/CNPJ:" in extract_text:
                    cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', extract_text)
                    if cpf_cnpj_formatado_match:
                        texto = cpf_cnpj_formatado_match.group(1)
                    
                        nf_dados_prestador['cpf_cnpj_com_mascara'] = texto
                        nf_dados_prestador['cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))
                else:
                    nf_dados_prestador['cpf_cnpj_com_mascara'] = 'None'
                    nf_dados_prestador['cpf_cnpj_sem_mascara'] = 'None'           
                        
                telefone_str = None
                #telefone_match = re.search(r'Telefone:\s+([0-9.\s-])', text)
                telefone_match = re.search(r'Telefone:\s+([0-9.\s-]+)', extract_text)
                if telefone_match: 
                    telefone_str = telefone_match.group(1)
                    # Remover quebras de linha
                    telefone_str = telefone_str.replace('.', '')
                    telefone_str = telefone_str.replace('\n', '')
                            
                    nf_dados_prestador['telefone'] = telefone_str
                else:
                    nf_dados_prestador['telefone'] = 'None'
                    
            # if father_to_process == "2_frame_cnpj_prestador":
            elif label == '2_frame_inscricao_prestador':
                coordinates = get_coordinates_filter(model_map, tipo, label, section)
                x0, y0, x1, y1 = coordinates[0]
                extract_text = extract_text_PIL(image_2work, (x0, y0, x1, y1))
                
                text_splited = texto_extraido_nf(extract_text)
                
                
                # Extrair Inscrição Estadual
                inscricao_estadual_match = re.search(r'Inscrição Estadual:\s+(.+)', extract_text)
                if inscricao_estadual_match:
                    inscricao_estadual_str = inscricao_estadual_match.group(1)
                    if inscricao_estadual_str == 'Nome/Razão Social:':
                        nf_inscr_estadual = 'None'
                        nf_dados_prestador['inscricao_estadual'] = "NONE"
                    else:
                        nf_inscr_estadual = inscricao_estadual_match.group(1)
                        nf_dados_prestador['inscricao_estadual'] = inscricao_estadual_match.group(1) 
                

            elif label == '2_frame_dados_prestador':
                dados_prestador = {}
                coordinates = get_coordinates_filter(model_map, tipo, label, section)
                x0, y0, x1, y1 = coordinates[0]
                
                extract_text = extract_text_PIL(image_2work, (x0, y0, x1, y1))
                dados_prestador = extract_fields_prestador(extract_text)
                
                text_splited = texto_extraido_nf(extract_text)
                keyword_list = ['Nome/Razão Social:', 'Nome de Fantasia:', 'Endereço:', 'E-mail:']
                string_pesquisa = "Nome/Razão Social:"
                texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            
                nf_dados_prestador['razao_social'] = texto

                string_pesquisa = "Nome de Fantasia:"
                texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                nf_dados_prestador['nome_fantasia'] = texto
                
                string_pesquisa = "Endereço:"
                texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                nf_dados_prestador['endereco'] = texto
                
                string_pesquisa = "E-mail:"
                texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                nf_dados_prestador['email'] = texto
                
    return nf_dados_prestador 

In [ ]:
def extrai_cnpj_R_PDF(texto_extraido, original_file_name):
    
    nf_values_prestador = {}
    message_erro = []
    try:
        keyword_list = ['CPF/CNPJ:', 'Telefone:']
        string_pesquisa = "CPF/CNPJ:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)

        cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', texto)
        cpf_cnpj_formatado_match
        if cpf_cnpj_formatado_match:
            nf_values_prestador['p_cpf_cnpj_com_mascara']= cpf_cnpj_formatado_match.group(1)
            nf_values_prestador['p_cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))
        print(cpf_cnpj_formatado, cpf_cnpj_sem_mascara) 
    
        string_pesquisa = "Telefone:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        telefone_match = re.search(r'Telefone:\s+([0-9.\s-]+)', texto)
        if telefone_match: 
            telefone_str = telefone_match.group(1)
            # Remover quebras de linha
            telefone_str = telefone_str.replace('.', '')
            telefone_str = telefone_str.replace('\n', '')
            
        else:
            telefone_str = 'None'
            
        nf_values_prestador['p_telefone'] = telefone_str    
            
    except Exception as e:
        msg = (f"doc: {original_file_name} | {e}")
        # nf_values_prestador['cpf_cnpj_com_mascara'] = 'None'
        # nf_values_prestador['cpf_cnpj_sem_mascara'] = 'None'
        # nf_values_prestador['telefone'] = 'None'
        # message_erro.append(msg)
    
    print(f'telefone_str: {telefone_str} | cpf_cnpj_formatado: {cpf_cnpj_formatado} | cpf_cnpj_sem_mascara: {cpf_cnpj_sem_mascara}')
    
    return nf_values_prestador  

In [ ]:
# 6. CNAE e Item da Lista de Serviços
def extrai_consiste_cnae_PDF_P(row, pdf_pesquisavel_map, de_para_pm, model_map, f_0_cnae, f_1_cnae, f_0_it, f_1_it, original_file_name, file_path):

    nf_data_CNAE = {}
    cnae_value = None
    item_servico_value = None
    
    nf_data_CNAE['Secao'] = "6. CNAE e Item da Lista de Serviços"

    process = ["cnae", 'item_lista_servicos']

    section = "6. CNAE e Item da Lista de Serviços"
    pdf_pesquisavel_map = True
        
    pdf_document = fitz.open(file_path)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]    
        
    tipo = "sframe_field"
    message_erro = []
    nf_dados_prestador = {}
    # model = row['model']
    for father in process:
        label = father
        if label == "cnae": 
            tipo = "sframe_field"
            #print(model_map)
            coordinates = get_coordinates_filter(pdf_pesquisavel_map=pdf_pesquisavel_map, model=model_map, tipo=tipo, label=label, section=section)
            x0, y0, x1, y1 = coordinates[0]
            y0 = y0 * f_0_cnae
            y1 = y1 * f_1_cnae
            text = page.get_text("text", clip=(x0, y0, x1, y1))
            text_extraido_cnae = text
            #print(f'model: {model_map} {"pdf_pesquisavel" if pdf_pesquisavel_map else "raster_pdf"}: labe: {label:>30} || x0:{x0:>6} | y0:{y0:>6} | x1:{x1:>6} | y1:{y1:>6}\n\n {text}')
            nf_data_CNAE_match = re.search(r'CNAE\s+(.+)', text)
            if nf_data_CNAE_match:
                # Remove a primeira ocorrência de "CNAE:"
                nf_data_CNAE_str = re.sub(r'^CNAE - ', '', text, count=1)
                # Remover quebras de linha
                nf_data_CNAE_str = nf_data_CNAE_str.replace('\n', ' ')
                
                cnae_number = int(extract_number(nf_data_CNAE_str))
                cnae_value = cnae_dict.get((de_para_pm, cnae_number),(0))
                if cnae_value != 0:
                    cnae_value = cnae_value.upper()
                    cnae_value = str(cnae_number) + " - " + cnae_value
                    #print(f' CNAE: {cnae_value} - Prefeitura: {de_para_pm}')
            
        
        elif label == "item_lista_servicos":
            tipo = "sframe_field"
            #print(model_map)

            coordinates = get_coordinates_filter(pdf_pesquisavel_map=pdf_pesquisavel_map, model=model_map, tipo=tipo, label=label, section=section)
            x0, y0, x1, y1 = coordinates[0]
            print(f'1. label: {label} | coordenadas originais: x0:{x0} y0:{y0} x1:{x1} y1:{y1} item: {original_file_name}')
            
            y0 = y0 * f_0_it
            y1 = y1 * f_1_it
            text = page.get_text("text", clip=(x0, y0, x1, y1))
            
            print(f'2. coordenadas ajustadas: x0:{x0} y0:{y0} x1:{x1} y1:{y1} item: {original_file_name} - text:\n{text}\n\n')
            
            #print(f'{text} - {label} x0:{x0} y0:{y0} x1:{x1} y1:{y1}')
            text_extraido_itens = text
            #print(f'model: {model_map} {"pdf_pesquisavel" if pdf_pesquisavel_map else "raster_pdf"}: labe: {label:>30} || x0:{x0:>6} | y0:{y0:>6} | x1:{x1:>6} | y1:{y1:>6}\n\n {text}')
            nf_item_lista_servicos_match = re.search(r'Item da Lista de Serviços\s+(.+)', text)
            print(f'1: nf_item_lista_servicos_match: {nf_item_lista_servicos_match}\n')
            if nf_item_lista_servicos_match:
                nf_item_lista_servicos_str = re.sub(r'^Item da Lista de Serviços - ', '', text, count=1)
                print(f'2: nf_item_lista_servicos_str: {nf_item_lista_servicos_str}\n') 
                item_servico_cod = float(extract_number(nf_item_lista_servicos_str))
                print(f'3: item_servico_cod: {item_servico_cod}\n') 
                item_servico_value, cnae_associado = item_servico_dict.get((de_para_pm, item_servico_cod), ("Valor não encontrado"))
                print(f'4: item_servico_value: {item_servico_value} | cnae_associado: {cnae_associado} \n') 
                item_servico_value = item_servico_value.upper()
                item_servico_value = str(item_servico_cod) + " - " + item_servico_value

    pdf_document.close()

    return cnae_value, item_servico_value    

In [ ]:
# 6C Processo de tratamento do CNAE anterior e sem pesquisa no dicionario
def processa_cnae_outros(text):
    
    nf_data_CNAE_match = re.search(r'CNAE\s+(.+)', text)
    if nf_data_CNAE_match:
        try:
            # Remove a primeira ocorrência de "CNAE:"
            nf_data_CNAE_str = re.sub(r'^CNAE - ', '', text, count=1)
            # Remover quebras de linha
            nf_data_CNAE_str = nf_data_CNAE_str.replace('\n', ' ')
            return nf_data_CNAE_str 
        except Exception as e:
            print(f"Erro busca cnae: {e}") 
        
    return None 

In [ ]:
# XXX Dados do Prestador de Servico
def processar_dados_prestador(row, pdf_pesquisavel_map, model_map, original_file_name, file_path):
    
    section = "2. PRESTADOR DE SERVIÇO"
    
    nf_dados_prestador = {}
    prestador_list = []
    f_tipo = 'frame'
    message_erro = []
    # if pdf_pesquisavel_map:
    
    nf_dados_prestador['pdf_pesquisavel'] = pdf_pesquisavel_map
    nf_dados_prestador['original_file_name'] = original_file_name
    
    data_box_valores = {'secao': section, 'erros': []}
   
    filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model_map) & (frames_nf_v4_df['section_json'] == section) & (frames_nf_v4_df['type'] == f_tipo)]
    
    if pdf_pesquisavel_map:
        pdf_document = fitz.open(file_path)
        page_number = 0
        page = pdf_document[page_number]
    else:
        image_2work, image_resized_name = convertResize(original_file_name, file_path, image_resized_path)

    for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
        
        frame_id = row_frame['id']
        label = row_frame['label']
        
        x0, y0, x1, y1 = (row_frame['x0_p'], row_frame['y0_p'], row_frame['x1_p'], row_frame['y1_p']) if pdf_pesquisavel_map else (row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1'])
        
        #print(f'doc {"pdf_pesquisavel_map" if pdf_pesquisavel_map else "raster_pdf"}: {label:>30} | id: {frame_id:>3} | x0: {x0:>6} y0: {y0:>6} x1: {x1:>6} y1: {y1:>6} ')
        
        if pdf_pesquisavel_map:
            texto_extraido = page.get_text("text", clip=(x0, y0, x1, y1))
            #data_box_valores['processo'] = "PDF_PESQUISAVEL"
            #texto_to_process = texto_pdf
        else:
            texto_extraido = extract_text_PIL(image_2work, (x0, y0, x1, y1))
            
            if label == '2_frame_cnpj_prestador':
                # Extrair CPF/CNPJ com máscara 1
                try:
                    
                    if "CPF/CNPJ:" in texto_extraido:
                        print(f'doc: {texto_extraido} pdf_pesqui: {pdf_pesquisavel_map} | {original_file_name}')
                        cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', texto_extraido)
                        if cpf_cnpj_formatado_match:
                            nf_dados_prestador['cpf_cnpj_com_mascara'] = cpf_cnpj_formatado_match.group(1)
                            nf_dados_prestador['cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))
                    else:
                        nf_dados_prestador['cpf_cnpj_com_mascara'] = 'None'
                        nf_dados_prestador['cpf_cnpj_sem_mascara'] = 'None'           
                            
                    telefone_str = None
                    #telefone_match = re.search(r'Telefone:\s+([0-9.\s-])', text)
                    telefone_match = re.search(r'Telefone:\s+([0-9.\s-]+)', texto_extraido)
                    if telefone_match: 
                        telefone_str = telefone_match.group(1)
                        # Remover quebras de linha
                        telefone_str = telefone_str.replace('.', '')
                        telefone_str = telefone_str.replace('\n', '')
                                
                        nf_dados_prestador['telefone'] = telefone_str
                    else:
                        nf_dados_prestador['telefone'] = 'None'
                except Exception as e:
                    msg = (f"doc: {original_file_name} | {e}")
                    nf_dados_prestador['cpf_cnpj_com_mascara'] = 'None'
                    nf_dados_prestador['cpf_cnpj_sem_mascara'] = 'None'
                    nf_dados_prestador['telefone'] = 'None'
                    message_erro.append(msg)       
            
            elif label == '2_frame_inscricao_prestador':
                #print(f'extract_fields_inscricao  {texto_extraido}')
                try:
                    print(f'doc: {texto_extraido} pdf_pesqui: {pdf_pesquisavel_map} | {original_file_name}')
                    text_splited = texto_extraido(texto_extraido)
                    keyword_list = ['Inscrição Municipal:', 'Inscrição Estadual:']
                    string_pesquisa = "Inscrição Municipal:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    nf_dados_prestador['inscricao_municipal'] = texto
                    
                    string_pesquisa = "Inscrição Estadual:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    nf_dados_prestador['inscricao_estadual'] = texto
                
                except Exception as e:
                    msg = (f"doc: {original_file_name} | {e}")
                    nf_dados_prestador['inscricao_municipal'] = " "
                    nf_dados_prestador['inscricao_estadual'] = " "
                    message_erro.append(msg)
            
            elif label == '2_frame_dados_prestador':
                # print(f' extract_fields_dados {texto_extraido}')
                try:
                    print(f'doc: {texto_extraido} pdf_pesqui: {pdf_pesquisavel_map} | {original_file_name}')
                    text_splited = texto_extraido(texto_extraido)
                    keyword_list = ['Nome/Razão Social:', 'Nome de Fantasia:', 'Endereço:', 'E-mail:']
                    string_pesquisa = "Nome/Razão Social:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    nf_dados_prestador['razao_social'] = texto

                    string_pesquisa = "Nome de Fantasia:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    nf_dados_prestador['nome_fantasia'] = texto
                    
                    string_pesquisa = "Endereço:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    nf_dados_prestador['endereco'] = texto
                    
                    string_pesquisa = "E-mail:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    nf_dados_prestador['email'] = texto
                    
                except Exception as e:
                    msg = (f"doc: {original_file_name} | {e}")
                    nf_dados_prestador['razao_social'] = " "
                    nf_dados_prestador['nome_fantasia'] = " "
                    nf_dados_prestador['endereco'] = " "
                    nf_dados_prestador['email'] = " "
                    message_erro.append(msg)
        prestador_list.append(nf_dados_prestador)
                
    if pdf_pesquisavel_map:
        pdf_document.close()
    
    print(nf_dados_prestador)    
        
    return prestador_list, message_erro

In [ ]:
def processar_prestador_raster_pdf(row, pdf_pesquisavel_map, model_map, original_file_name, file_path):    
    
    section = "2. PRESTADOR DE SERVIÇO"
    pdf_pesquisavel_map = False
    
    tipo= "frame"
    message_erro = []
    nf_dados_prestador = {}
    
    process_prestador = ['2_frame_cnpj_prestador', '2_frame_inscricao_prestador', '2_frame_dados_prestador']

    image_2work, image_resized_name = convertResize(original_file_name, file_path, image_resized_path)
    
    for father in process_prestador:
        father_to_process = father
        if father_to_process == "2_frame_cnpj_prestador":
            
            
            x0, y0, x1, y1 = (row_frame['x0_p'], row_frame['y0_p'], row_frame['x1_p'], row_frame['y1_p']) if pdf_pesquisavel_map else (row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1'])
            texto_extraido = extract_text_PIL(image_2work, (x0, y0, x1, y1))
            
            data_prestador = {}
            f_father = father_to_process
            section = "2. PRESTADOR DE SERVIÇO"
            
            filtered_frame_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model_map) & (frames_nf_v4_df['label'] == father_to_process) & (frames_nf_v4_df['type'] == tipo)]
            
            Texto_extraido = extract_dados_from_frame(model, f_father, section)
            print(Texto_extraido)
        
        
        elif frame_pesquisa == "2_frame_cnpj_prestador":
        data_prestador = {}
        f_father = "2_frame_cnpj_prestador"
        section = "2. PRESTADOR DE SERVIÇO"
        Texto_extraido = extract_dados_from_frame(model, f_father, section)
        print(Texto_extraido)
        
        print(father_to_process)
        for index_frame, row_frame in filtered_frame_nf_v4_df.iterrows():
            label_value = row_frame['label']
            reference_value = row_frame['reference']
            f_type = row_frame['type']
            frame_id = row_frame['id']
            
            if label_value == '2_frame_cnpj_prestador':
                
                x0, y0, x1, y1 = (row_frame['x0_p'], row_frame['y0_p'], row_frame['x1_p'], row_frame['y1_p']) if pdf_pesquisavel_map else (row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1'])
                texto_extraido = extract_text_PIL(image_2work, (x0, y0, x1, y1))
                
                print(f'label: {label_value:>10}  {x0}, {y0}, {x1}, {y1} | ')
                # Extrair CPF/CNPJ com máscara 1
                try:
                    if "CPF/CNPJ:" in texto_extraido:
                        print(f'doc: {texto_extraido} pdf_pesqui: {pdf_pesquisavel_map} | {original_file_name}')
                        cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', texto_extraido)
                        if cpf_cnpj_formatado_match:
                            nf_dados_prestador['cpf_cnpj_com_mascara'] = cpf_cnpj_formatado_match.group(1)
                            nf_dados_prestador['cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))
                    else:
                        nf_dados_prestador['cpf_cnpj_com_mascara'] = 'None'
                        nf_dados_prestador['cpf_cnpj_sem_mascara'] = 'None'           
                            
                    telefone_str = None
                    #telefone_match = re.search(r'Telefone:\s+([0-9.\s-])', text)
                    telefone_match = re.search(r'Telefone:\s+([0-9.\s-]+)', texto_extraido)
                    if telefone_match: 
                        telefone_str = telefone_match.group(1)
                        # Remover quebras de linha
                        telefone_str = telefone_str.replace('.', '')
                        telefone_str = telefone_str.replace('\n', '')
                                
                        nf_dados_prestador['telefone'] = telefone_str
                    else:
                        nf_dados_prestador['telefone'] = 'None'
                except Exception as e:
                    msg = (f"doc: {original_file_name} | {e}")
                    nf_dados_prestador['cpf_cnpj_com_mascara'] = 'None'
                    nf_dados_prestador['cpf_cnpj_sem_mascara'] = 'None'
                    nf_dados_prestador['telefone'] = 'None'
                    message_erro.append(msg) 
                     
            elif label_value == '2_frame_inscricao_prestador':
                
                x0, y0, x1, y1 = (row_frame['x0_p'], row_frame['y0_p'], row_frame['x1_p'], row_frame['y1_p']) if pdf_pesquisavel_map else (row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1'])
                texto_extraido = extract_text_PIL(image_2work, (x0, y0, x1, y1))
                
                #print(f'extract_fields_inscricao  {texto_extraido}')
                try:
                    print(f'doc: {texto_extraido} pdf_pesqui: {pdf_pesquisavel_map} | {original_file_name}')
                    text_splited = texto_extraido(texto_extraido)
                    keyword_list = ['Inscrição Municipal:', 'Inscrição Estadual:']
                    string_pesquisa = "Inscrição Municipal:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    nf_dados_prestador['inscricao_municipal'] = texto
                    
                    string_pesquisa = "Inscrição Estadual:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    nf_dados_prestador['inscricao_estadual'] = texto
                
                except Exception as e:
                    msg = (f"doc: {original_file_name} | {e}")
                    nf_dados_prestador['inscricao_municipal'] = " "
                    nf_dados_prestador['inscricao_estadual'] = " "
                    message_erro.append(msg)
            
            
            elif label_value == '2_frame_dados_prestador':
                # print(f' extract_fields_dados {texto_extraido}')
                x0, y0, x1, y1 = (row_frame['x0_p'], row_frame['y0_p'], row_frame['x1_p'], row_frame['y1_p']) if pdf_pesquisavel_map else (row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1'])
                texto_extraido = extract_text_PIL(image_2work, (x0, y0, x1, y1))
                
                try:
                    print(f'doc: {texto_extraido} pdf_pesqui: {pdf_pesquisavel_map} | {original_file_name}')
                    text_splited = texto_extraido(texto_extraido)
                    keyword_list = ['Nome/Razão Social:', 'Nome de Fantasia:', 'Endereço:', 'E-mail:']
                    string_pesquisa = "Nome/Razão Social:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    nf_dados_prestador['razao_social'] = texto

                    string_pesquisa = "Nome de Fantasia:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    nf_dados_prestador['nome_fantasia'] = texto
                    
                    string_pesquisa = "Endereço:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    nf_dados_prestador['endereco'] = texto
                    
                    string_pesquisa = "E-mail:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    nf_dados_prestador['email'] = texto
                    
                except Exception as e:
                    msg = (f"doc: {original_file_name} | {e}")
                    nf_dados_prestador['razao_social'] = " "
                    nf_dados_prestador['nome_fantasia'] = " "
                    nf_dados_prestador['endereco'] = " "
                    nf_dados_prestador['email'] = " "
                    message_erro.append(msg)
        #prestador_list.append(nf_dados_prestador)
                

 
        
    return nf_dados_prestador

In [ ]:
def processar_prestador_raster_pdf(row, pdf_pesquisavel_map, model_map, original_file_name, file_path):
    
    section = "2. PRESTADOR DE SERVIÇO"
    pdf_pesquisavel_map = False
    
    tipo= "frame"
    message_erro = []
    nf_dados_prestador = {}
    
    process_prestador = ['2_frame_cnpj_prestador', '2_frame_inscricao_prestador', '2_frame_dados_prestador']

    image_2work, image_resized_name = convertResize(original_file_name, file_path, image_resized_path)
    
    for father in process_prestador:
        father_to_process = father
        
        filtered_frame_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model_map) & (frames_nf_v4_df['label'] == father_to_process) & (frames_nf_v4_df['type'] == tipo)]
        print(father_to_process)
        for index_frame, row_frame in filtered_frame_nf_v4_df.iterrows():
            label_value = row_frame['label']
            reference_value = row_frame['reference']
            f_type = row_frame['type']
            frame_id = row_frame['id']
            
            if label_value == '2_frame_cnpj_prestador':
                
                x0, y0, x1, y1 = (row_frame['x0_p'], row_frame['y0_p'], row_frame['x1_p'], row_frame['y1_p']) if pdf_pesquisavel_map else (row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1'])
                texto_extraido = extract_text_PIL(image_2work, (x0, y0, x1, y1))
                
                print(f'label: {label_value:>10}  {x0}, {y0}, {x1}, {y1} | ')
                # Extrair CPF/CNPJ com máscara 1
                try:
                    if "CPF/CNPJ:" in texto_extraido:
                        print(f'doc: {texto_extraido} pdf_pesqui: {pdf_pesquisavel_map} | {original_file_name}')
                        cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', texto_extraido)
                        if cpf_cnpj_formatado_match:
                            nf_dados_prestador['cpf_cnpj_com_mascara'] = cpf_cnpj_formatado_match.group(1)
                            nf_dados_prestador['cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))
                    else:
                        nf_dados_prestador['cpf_cnpj_com_mascara'] = 'None'
                        nf_dados_prestador['cpf_cnpj_sem_mascara'] = 'None'           
                            
                    telefone_str = None
                    #telefone_match = re.search(r'Telefone:\s+([0-9.\s-])', text)
                    telefone_match = re.search(r'Telefone:\s+([0-9.\s-]+)', texto_extraido)
                    if telefone_match: 
                        telefone_str = telefone_match.group(1)
                        # Remover quebras de linha
                        telefone_str = telefone_str.replace('.', '')
                        telefone_str = telefone_str.replace('\n', '')
                                
                        nf_dados_prestador['telefone'] = telefone_str
                    else:
                        nf_dados_prestador['telefone'] = 'None'
                except Exception as e:
                    msg = (f"doc: {original_file_name} | {e}")
                    nf_dados_prestador['cpf_cnpj_com_mascara'] = 'None'
                    nf_dados_prestador['cpf_cnpj_sem_mascara'] = 'None'
                    nf_dados_prestador['telefone'] = 'None'
                    message_erro.append(msg) 
                     
            elif label_value == '2_frame_inscricao_prestador':
                
                x0, y0, x1, y1 = (row_frame['x0_p'], row_frame['y0_p'], row_frame['x1_p'], row_frame['y1_p']) if pdf_pesquisavel_map else (row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1'])
                texto_extraido = extract_text_PIL(image_2work, (x0, y0, x1, y1))
                
                #print(f'extract_fields_inscricao  {texto_extraido}')
                try:
                    print(f'doc: {texto_extraido} pdf_pesqui: {pdf_pesquisavel_map} | {original_file_name}')
                    text_splited = texto_extraido(texto_extraido)
                    keyword_list = ['Inscrição Municipal:', 'Inscrição Estadual:']
                    string_pesquisa = "Inscrição Municipal:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    nf_dados_prestador['inscricao_municipal'] = texto
                    
                    string_pesquisa = "Inscrição Estadual:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    nf_dados_prestador['inscricao_estadual'] = texto
                
                except Exception as e:
                    msg = (f"doc: {original_file_name} | {e}")
                    nf_dados_prestador['inscricao_municipal'] = " "
                    nf_dados_prestador['inscricao_estadual'] = " "
                    message_erro.append(msg)
            
            
            elif label_value == '2_frame_dados_prestador':
                # print(f' extract_fields_dados {texto_extraido}')
                x0, y0, x1, y1 = (row_frame['x0_p'], row_frame['y0_p'], row_frame['x1_p'], row_frame['y1_p']) if pdf_pesquisavel_map else (row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1'])
                texto_extraido = extract_text_PIL(image_2work, (x0, y0, x1, y1))
                
                try:
                    print(f'doc: {texto_extraido} pdf_pesqui: {pdf_pesquisavel_map} | {original_file_name}')
                    text_splited = texto_extraido(texto_extraido)
                    keyword_list = ['Nome/Razão Social:', 'Nome de Fantasia:', 'Endereço:', 'E-mail:']
                    string_pesquisa = "Nome/Razão Social:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    nf_dados_prestador['razao_social'] = texto

                    string_pesquisa = "Nome de Fantasia:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    nf_dados_prestador['nome_fantasia'] = texto
                    
                    string_pesquisa = "Endereço:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    nf_dados_prestador['endereco'] = texto
                    
                    string_pesquisa = "E-mail:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    nf_dados_prestador['email'] = texto
                    
                except Exception as e:
                    msg = (f"doc: {original_file_name} | {e}")
                    nf_dados_prestador['razao_social'] = " "
                    nf_dados_prestador['nome_fantasia'] = " "
                    nf_dados_prestador['endereco'] = " "
                    nf_dados_prestador['email'] = " "
                    message_erro.append(msg)
        #prestador_list.append(nf_dados_prestador)
                

 
        

In [ ]:
# XXX Dados do Prestador de Servico
def processar_dados_prestador_raster_pdf(row, pdf_pesquisavel_map, model_map, original_file_name, file_path):
    
    section = "2. PRESTADOR DE SERVIÇO"
    
    
    prestador_list = []
    f_tipo = 'frame'
    message_erro = []
    # if pdf_pesquisavel_map:
    data_box_valores = {'secao': section, 'erros': []}
   
    filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model_map) & (frames_nf_v4_df['section_json'] == section) & (frames_nf_v4_df['type'] == f_tipo)]
    
    
        
  
     
    else:
        image_2work, image_resized_name = convertResize(original_file_name, file_path, image_resized_path)

    for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
        
        frame_id = row_frame['id']
        label = row_frame['label']
        
        x0, y0, x1, y1 = (row_frame['x0_p'], row_frame['y0_p'], row_frame['x1_p'], row_frame['y1_p']) if pdf_pesquisavel_map else (row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1'])
        
        #print(f'doc {"pdf_pesquisavel_map" if pdf_pesquisavel_map else "raster_pdf"}: {label:>30} | id: {frame_id:>3} | x0: {x0:>6} y0: {y0:>6} x1: {x1:>6} y1: {y1:>6} ')
        
        if pdf_pesquisavel_map:
            texto_extraido = page.get_text("text", clip=(x0, y0, x1, y1))
            #data_box_valores['processo'] = "PDF_PESQUISAVEL"
            #texto_to_process = texto_pdf
        else:
            texto_extraido = extract_text_PIL(image_2work, (x0, y0, x1, y1))
            
    return texto_extraido 

In [ ]:
def extract_fields_cnpj(texto_extraido, original_file_name): 
  
    message_erro = []
    
    data_nf_cnpj_prestador = {}
      
    print(f'2 extract_fields_cnpj:  {texto_extraido}')
    
    # Extrair CPF/CNPJ com máscara 1
    if "CPF/CNPJ:" in texto_extraido:
        cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', texto_extraido)
        if cpf_cnpj_formatado_match:
            data_nf_cnpj_prestador['cpf_cnpj_com_mascara'] = cpf_cnpj_formatado_match.group(1)
            data_nf_cnpj_prestador['cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))
    else:
        data_nf_cnpj_prestador['cpf_cnpj_com_mascara'] = None
        data_nf_cnpj_prestador['cpf_cnpj_sem_mascara'] = None            
            
            
    telefone_str = None

    #telefone_match = re.search(r'Telefone:\s+([0-9.\s-])', text)
    telefone_match = re.search(r'Telefone:\s+([0-9.\s-]+)', texto_extraido)
    if telefone_match: 
        telefone_str = telefone_match.group(1)
        # Remover quebras de linha
        telefone_str = telefone_str.replace('.', '')
        telefone_str = telefone_str.replace('\n', '')
                
        data_nf_cnpj_prestador['telefone'] = telefone_str
    else:
        data_nf_cnpj_prestador['telefone'] = None 
    
    #print(data_nf_cnpj_prestador)    

             
    return data_nf_cnpj_prestador


def extract_fields_inscricao(texto_extraido, original_file_name):
    
    data_nf_incricao_prestador = {} 
    message_erro = []
    
    print(f'extract_fields_inscricao  {texto_extraido}')
    try:
        text_splited = texto_extraido(texto_extraido)
        keyword_list = ['Inscrição Municipal:', 'Inscrição Estadual:']
        string_pesquisa = "Inscrição Municipal:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        data_nf_incricao_prestador['inscricao_municipal'] = texto
        
        string_pesquisa = "Inscrição Estadual:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        data_nf_incricao_prestador['inscricao_estadual'] = texto
      
    except Exception as e:
        msg = (f"doc: {original_file_name} | {e}")
        data_nf_incricao_prestador['inscricao_municipal'] = " "
        data_nf_incricao_prestador['inscricao_estadual'] = " "
        message_erro.append(msg)
        
    #print(data_nf_incricao_prestador)   
    
    return data_nf_incricao_prestador


def extract_fields_dados(texto_extraido, original_file_name):
    
    data_nf_dados_prestador = {}
    message_erro = []
    print(f' extract_fields_dados {texto_extraido}')
    try:

        keyword_list = ['Nome/Razão Social:', 'Nome de Fantasia:', 'Endereço:', 'E-mail:']
        
        string_pesquisa = "Nome/Razão Social:"
        text_splited = texto_extraido(texto_extraido)
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        data_nf_dados_prestador['razao_social'] = texto

        string_pesquisa = "Nome de Fantasia:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        data_nf_dados_prestador['nome_fantasia'] = texto
        
        string_pesquisa = "Endereço:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        data_nf_dados_prestador['endereco'] = texto
        
        string_pesquisa = "E-mail:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        data_nf_dados_prestador['email'] = texto
        
    except Exception as e:
        msg = (f"doc: {original_file_name} | {e}")
        data_nf_dados_prestador['razao_social'] = " "
        data_nf_dados_prestador['nome_fantasia'] = " "
        data_nf_dados_prestador['endereco'] = " "
        data_nf_dados_prestador['email'] = " "
        message_erro.append(msg)
        
    #print(data_nf_dados_prestador)
    
    return data_nf_dados_prestador 

In [ ]:
# 4. XXX Pesquisa prefeitura no documento (dando as coordenadas) e efetuando o OCR
def pequisaTextoDoc(image_name):

    # 1. Definindo as coordenadas do frame
    x0 = 0
    y0 = 0
    x1= 2066
    y1 = 2922

    # 2. Definir frame_image
    frame_image = image_name.crop((x0, y0, x1, y1))

    # 3. Extraia texto usando OCR com configuração de idioma padrão para este frame
    extracted_text_frame = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config).strip()
    
    return extracted_text_frame

In [ ]:
# 2. XXX Analisa nro de paginas
def analisa_nro_pages(file_path):
    
    pdf_document = fitz.open(file_path)
    pages = pdf_document.pages() # generator object

    page_nro = []
    for page in pages:
        page_nro.append(page)
        
    nro_paginas = len(page_nro)    
    if nro_paginas > 1:
        doc_1_page = False
        return doc_1_page, nro_paginas    
    else:
        doc_1_page = True
        return doc_1_page, nro_paginas  
    pdf_document.close()
    



# XXX FUNCAO DE SPLIT
def split_documentos(qualquer_df, outro_tgt_df, num_linhas_df, fase, atividade, status):
    
    documentos_splitados = []
    doc_info = {}
    rows_list = []
    documentos = []
    #output_dir = os.path.join(documentos_scan_path, batch_name)

    fase_processo = fase
    atividade_processo = atividade
    status_documento = status
    
    time_now = cron.timenow_pt_BR()
    
    i = num_linhas_df + 1
    for idx, row in qualquer_df.iterrows():
        message_erro = []
        batch_name = row['batch']
        original_file_name = row['original_file_name']
        folder_name = row['directory']
        file_path = row['file_path']
        level = row['level']
        d_type = row['type']
        document_type = row['document_type']
        # document_unique_id = row['document_unique_id']
        new_level = level + 1
        try:
            pdf = fitz.open(file_path)
            # Número total de páginas no PDF
            total_pages = len(pdf)
        except Exception as e:
            print(f"Nao congui abrir o PDF: {e}")    

        # Nome base para os arquivos de saída
        base_name = file_path.split('.')[0]  # Remove a extensão do arquivo
        
        file_to_delete = file_path

        # Loop para criar um novo PDF para cada página
        for page_num in range(total_pages):
            # Cria um novo objeto PDF
            new_pdf = fitz.open()
            # Adiciona a página atual ao novo PDF
            new_pdf.insert_pdf(pdf, from_page=page_num, to_page=page_num)
            # Nome do novo arquivo PDF
            new_pdf_name = f"{base_name}_page_{page_num + 1}.pdf"
            # Salva o novo PDF
            new_pdf.save(new_pdf_name)
            # Fecha o novo PDF
            new_pdf.close()
            status_documento = 'splitado'
            name_pdf_splited = os.path.basename(new_pdf_name)
            
            nova_linha = {
                'seq': i,
                'date_time': time_now,
                'batch': batch_name,
                'fase_processo': fase_processo,
                'nome_atividade': atividade_processo,
                'status_documento': status_documento,
                'document_unique_id': generate_unique_id(),
                'original_file_name': name_pdf_splited,
                'directory': folder_name,
                'one_page': True,
                'pages': 1,                    
                'level': new_level,
                'type': d_type,
                'document_type': document_type,
                'parent_document_unique_id': idx,
                'file_path': new_pdf_name,
                'file_hash': generate_file_hash(new_pdf_name),
            }
            rows_list.append(nova_linha)
            i += 1
        outro_tgt_df.loc[idx, 'status_documento'] = "NAO_PROCESSAR"    
            

    total_split = i - 1
                      
      
    df_split = pd.DataFrame(rows_list)
    
    return df_split


# 1.XXX  Acao 1 - Ler todo o pipeline de documentos recebidos
def scan_pipeline_documentos(documentos_scan_path, batch_name, fase, atividade, status):

    doc_info = {}
    resumo = {}
    documentos = []
    output_dir = os.path.join(documentos_scan_path, batch_name)

    fase_processo = fase
    atividade_processo = atividade
    status_documento = status
    
    time_now = cron.timenow_pt_BR()
    
    #print(f'1. Fase atual: {fase_atual} | Nome atividade: {nome_atividade} | Status documento: {status_documento}')
    
    # Just for DEV
    parent_document_unique_id = generated_parent_document_unique_id

    i = 1
    for root, dirs, files in os.walk(output_dir):
        folder_name = os.path.basename(root)
        #print(folder_name)
        for file in files:
            level = 3
            file_path = os.path.join(root, file)
            new_path_name = os.path.join(output_dir, file)
            diretorio = os.path.basename(file_path)
            if folder_name == batch_name:
                folder_name = "root_dir"
            doc_one_page, nro_pgs = analisa_nro_pages(file_path)
            #print(f'2. doc_one_page: {doc_one_page} | nro_pgs: {nro_pgs}') 
            type = 'document_file'
            document_type = "provavel_NFSe"
            #print(f'\nfile: {file} | diretorio: {folder_name}\n{file_path} ')
            if folder_name not in doc_info:
                doc_info[folder_name] = {"data": []}
            doc_info[folder_name]["data"].append({
                "seq": i,
                "date_time": time_now,
                "batch": batch_name,
                "fase_processo": fase_processo,
                "nome_atividade": atividade_processo,
                "original_file_name": file,
                "status_documento": status_documento,
                "one_page": doc_one_page,
                "pages": nro_pgs,
                "directory": folder_name,
                "file_path": file_path,
                "level": level,
                "type": type,
                "document_type": document_type,
                "document_unique_id": generate_unique_id(),
                "parent_document_unique_id": parent_document_unique_id,
                "file_hash": generate_file_hash(file_path)
            })

            i += 1

    
    documentos.append(doc_info)
    total_scan = i - 1
    #print("Documentos scaneados: ", total_scan)
    
    # Criaçao do df_scan_pipe
    document_row = {}
    rows_list = []
    for documento in documentos:
        #print(documento)
        for diretorio, dados in documento.items():
            #print(f'Diretório: {diretorio}\n')
            for arquivo in dados['data']:
                nova_linha = {
                    'seq': arquivo['seq'],
                    'date_time': arquivo['date_time'],
                    'batch': arquivo['batch'],
                    'fase_processo': arquivo['fase_processo'],
                    'nome_atividade': arquivo['nome_atividade'],
                    'status_documento': arquivo['status_documento'],
                    'document_unique_id': arquivo["document_unique_id"],
                    'original_file_name': arquivo['original_file_name'],
                    'directory': arquivo['directory'],
                    'one_page': arquivo['one_page'],
                    'pages': arquivo['pages'],
                    'pdf_pesquisavel': None,
                    'prefeitura': None,
                    'cnpj': None,
                    'modelo': None,
                    'level': arquivo['level'],
                    'type': arquivo['type'],
                    'document_type': arquivo['document_type'],
                    'parent_document_unique_id': arquivo['parent_document_unique_id'],
                    'file_path': arquivo["file_path"],
                    'file_hash': arquivo["file_hash"],
                }
                rows_list.append(nova_linha)
    
    df_trans_pipe = pd.DataFrame(rows_list)            
    
    return df_trans_pipe, documentos



    
    
#generated_parent_document_unique_id = generate_unique_id()  

# Processo de deleçao e atualizacao de documentos
#e_deleta_peloamor(df_docs_splitados)

#me_atualiza_logo_vai_2(novo_df)  

In [ ]:
def confirma_pdf_pequisavel(file_path):
    
    pdf_document = fitz.open(file_path)
    # Página do PDF  ATENCAO  (UNICA PAGINA)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]
    # Definir retângulo de interesse
    x0 = 0
    y0 = 4
    x1 = 600
    y1 = 200  # Ajuste este valor para delimitar a região vertical
    # Extrair texto dentro do retângulo
    text = page.get_text("text", clip=(x0, y0, x1, y1))
    if text:
       page_number = 0
       pdf_realmente_pequisavel = True
       #print(page_number)
    else:
       page_number = 1
       pdf_realmente_pequisavel = False
       #print(page_number)
    
    try:
        page = pdf_document[page_number]
        x0 = 0
        y0 = 0
        x1 = 600
        y1 = 110

        text = page.get_text("text", clip=(x0, y0, x1, y1))
        nf_data_cabecalho = Extc.extract_fields_cabecalho(text)
        
        if nf_data_cabecalho:
            pdf_realmente_pequisavel = True
        else:
            pdf_realmente_pequisavel = False
            
      
        return pdf_realmente_pequisavel, page_number
    except Exception as e:
        print(f"Erro ao abrir pagina do PDF: {e}")
        return pdf_realmente_pequisavel, page_number
    pdf_document.close()  

In [ ]:
def remove_images(imagens_list):
    for image in imagens_list:
        try:
            os.remove(image)
        except Exception as e:
            print(f"Nao congui deletar: {e}")

# Demais funcoes para guarda e analise

In [ ]:
def pesquisa_prefeitura_pdf_pesquisavel(file_path):    
    
    # Carregar o arquivo PDF
    pdf_document = fitz.open(file_path)

    # Página do PDF  ATENCAO  (UNICA PAGINA)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]

    # Definir retângulo de interesse
    x0 = 0
    y0 = 4
    x1 = 600
    y1 = 200  # Ajuste este valor para delimitar a região vertical

    # Extrair texto dentro do retângulo
    text = page.get_text("text", clip=(x0, y0, x1, y1))
    
    #print(text)
    
    if text:
       page_number = 0
       #print(page_number)
    else:
       page_number = 1
       #print(page_number)
    
    pdf_document.close()
   
    return text

In [ ]:
def confirma_pdf_pequisavel3(file_path):
    
    pdf_document = fitz.open(file_path)
    # Página do PDF  ATENCAO  (UNICA PAGINA)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]
    # Definir retângulo de interesse
    x0 = 0
    y0 = 4
    x1 = 600
    y1 = 200  # Ajuste este valor para delimitar a região vertical
    # Extrair texto dentro do retângulo
    text = page.get_text("text", clip=(x0, y0, x1, y1))
    if text:
       page_number = 0
       pdf_realmente_pequisavel = True
       #print(page_number)
    else:
       page_number = 1
       pdf_realmente_pequisavel = False
       #print(page_number)
    
    try:
        page = pdf_document[page_number]
        x0 = 0
        y0 = 0
        x1 = 600
        y1 = 110

        text = page.get_text("text", clip=(x0, y0, x1, y1))
        nf_data_cabecalho = Extc.extract_fields_cabecalho(text)
        
        if nf_data_cabecalho:
            pdf_realmente_pequisavel = True
        else:
            pdf_realmente_pequisavel = False
            
      
        return pdf_realmente_pequisavel, page_number
    except Exception as e:
        print(f"Erro ao abrir pagina do PDF: {e}")
        return pdf_realmente_pequisavel, page_number
    pdf_document.close()   

In [ ]:
# Funcao deletar documentos pelo DF
def me_deleta_peloamor(df_deleta):
    
    for idx, row in df_deleta.iterrows():
        file_delete_path = row['file_path']
        try:
            os.remove(file_delete_path)
        except Exception as e:
            print(f"Nao congui deletar: {e}")   

In [ ]:
# Funcao deletar documentos pelo DF
def me_atualiza_logo_vai_2(qualquer_df):
    
    for idx, row in qualquer_df.iterrows():
        file_delete_path = row['file_path']
        try:
            df_temp_pipe.loc[idx, 'status_documento'] = "NAO PROCESSAR"
            
        except Exception as e:
            print(f"Nao congui atualizar: {e}")

In [ ]:
def convertResize(doc2convert, document_path, image_resized_path):
    
    
    name_image = conv_filename_no_ext(doc2convert)
    
    # # 1. remocao do sufixo .pdf
    # if doc2convert.split(".")[1].islower():
    #     nameImage= doc2convert.removesuffix(".pdf")
    # else:
    #     nameImage= doc2convert.removesuffix(".PDF")
        
    # print(f'nameImage: {nameImage}\n')    
    
    # 2. construo um novo nome para o documento imagem
    image_resized_name = os.path.join(f'{image_resized_path}/{str(name_image)}.jpg')
    #print(f'image_resized_name: {image_resized_name}\n')
    # 3. Conversao para imagem
    pages = convert_from_path(document_path, 500, poppler_path=poppler_path)
    
    # 4. Verifica se ha mais que uma pagina
    if len(pages) > 1:
        raise ValueError("Erro, documento com mais de uma página")
    else:
        # 5. Iterar pelas páginas e redimensionar
        resized_pages = []
        for page in pages:
            resized_page = page.resize((2067, 2923))
            resized_pages.append(resized_page)
            
        resized_pages[0].save(image_resized_name, 'JPEG')
        
        image_2work = resized_pages[0]
        
    return image_2work, image_resized_name


# 1. converte nome do arquivo
def conv_filename(title):
    
    # Divide o título em nome e extensão
    name, extension = title.rsplit('.', 1) if '.' in title else (title, "")

    # Remove acentos e caracteres especiais do nome
    name = normalize('NFKD', name).encode('ASCII', 'ignore').decode('ASCII')
    
    # Substiti espaços e hífens por sublinhados
    filename = name.replace(' ', '_').replace('-', '_')

    # Remove quaisquer outros caracteres não alfanuméricos, exceto sublinhados
    filename = re.sub(r'[^\w_]', '', filename)

    # Converte para minúsculas
    filename = filename.lower()

    # Adiciona a extensão de volta, se houver
    if extension:
        filename += '.' + extension.lower()

    return filename

def conv_filename_no_ext(title):
    
    # Divida o título em nome e extensão (mas ignore a extensão)
    name = title.rsplit('.', 1)[0] if '.' in title else title

    # Remova acentos e caracteres especiais do nome
    name = normalize('NFKD', name).encode('ASCII', 'ignore').decode('ASCII')
    
    # Substitua espaços e hífens por sublinhados
    filename = name.replace(' ', '_').replace('-', '_')

    # Remova quaisquer outros caracteres não alfanuméricos, exceto sublinhados
    filename = re.sub(r'[^\w_]', '', filename)

    # Converter para minúsculas
    filename = filename.lower()

    return filename 

## Funcoes para validar 

In [ ]:
# 2. Pesquisa prefeitura no documento
def pequisa_model_raster_pdf(image_name):

    # 1. Definindo as coordenadas do frame
    x0 = 406
    y0 = 0
    x1= 1540
    y1 = 380

    # 2. Definir frame_image
    frame_image = image_name.crop((x0, y0, x1, y1))

    # 3. Extraia texto usando OCR com configuração de idioma padrão para este frame
    extracted_text_frame = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config).strip()

    # 4. Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
    values = extracted_text_frame.split('\n')

    # 5. Interacao para pesquisar prefeitura
    for value in values:
        nome_prefeitura_match = re.search(r'PREFEITURA (.+)', value)
        if nome_prefeitura_match:
            nome_prefeitura = "PREFEITURA " + nome_prefeitura_match.group(1) 
            return  nome_prefeitura  

In [ ]:
def processar_cabecalho(row, model_map, original_file_name, file_path, frame_type):
    
    
    section = "1 - CABECALHO"
    f_frame_label = "1_frame_dados_nf"
    data_box_valores = {'secao': section}
    nf_data_cabecalho = {}
    
    section = "1 - CABECALHO"
    f_tipo = 'frame'
    f_frame_label = "1_frame_dados_nf"
    data_extrated_prefeitura = {}
    
    
    1_frame_dados_nf
    
 
    message_erro = []
    
    filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model_map) & (frames_nf_v4_df['section_json'] == section)]
    
    image_2work, image_resized_name = convertResize(original_file_name, file_path, image_resized_path)

    for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
        
        if row_frame['father'] == "1_frame_prefeitura_nf":
        
        if row_frame['label'] == "1_frame_dados_nf":
            
            
            
            tipo = "sframe_field"
            data_extrated_prefeitura = {}
   

            filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model) & (frames_nf_v4_df['father'] == father) & (frames_nf_v4_df['type'] == tipo)]

            for index_sframe, row_sframe in filtered_frames_nf_v4_df.iterrows():
                
                label_value = row_sframe['label']
                
                #print("label_value", label_value)
                
                if label_value == "nome_prefeitura":
                    reference_value = row_sframe['reference']
                    for value in values:
                        result = process_line(value, reference_value, label_value)
                        if result:
                            data_extrated_prefeitura.update(result)
                elif label_value == "secretaria":
                    reference_value = row_sframe['reference']
                    for value in values:
                        result = process_line(value, reference_value, label_value)
                        if result:
                            data_extrated_prefeitura.update(result) 
                elif label_value == "tipo_nota_fiscal":
                    reference_value = row_sframe['reference']  
                    for value in values:
                        result = process_line(value, reference_value, label_value)
                        if result:
                            data_extrated_prefeitura.update(result)
                            
            return data_extrated_prefeitura
            
        
        frame_id = row_frame['id']
            label = row_frame['label']
            x0, y0, x1, y1 = (row_frame['x0_p'], row_frame['y0_p'], row_frame['x1_p'], row_frame['y1_p']) if pdf_pesquisavel else (row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1'])
        
        
        #print(f'doc {"pdf_pesquisavel" if pdf_pesquisavel else "raster_pdf"}: {label:>30} | id: {frame_id:>3} | x0: {x0:>6} y0: {y0:>6} x1: {x1:>6} y1: {y1:>6} ')
      
        
        text_splited = texto_extraido_cabecalho(texto_extraido)
        keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']

        string_pesquisa = "Número da Nota:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)         
        nro_nota_frame = texto
        #print(nro_nota_frame)
        nf_data_cabecalho['numero_nota_fiscal'] = nro_nota_frame
        
        string_pesquisa = "Competência:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        competencia_frame = texto
        nf_data_cabecalho['competencia'] = competencia_frame 
        #print(competencia_frame)
        
        string_pesquisa = "ata e Hora da Emissão:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        dt_hr_emissao_frame = texto
        nf_data_cabecalho['dt_hr_emissao'] = dt_hr_emissao_frame 
        
        string_pesquisa = "Código Verificação:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        codigo_ver_frame = texto
        nf_data_cabecalho['codigo_verificacao'] = codigo_ver_frame 
            
            
            

            
            

    
    # if pdf_pesquisavel:
    #     pdf_document.close()
        
    return nf_data_cabecalho



def cabecalho_prefeitura():
    valor_dict = {}
    dados_prefeitura = {}
    f_frame_name = "1_frame_prefeitura_nf"
    # 1. funçao basica de modelo 
    texto = executa_model_frame(model, secao, f_frame_name)
    text_splited = texto.split('\n')
    
    valor_dict = extract_prefeitura(model, f_frame_name, text_splited)
    if valor_dict:
        dados_prefeitura.update(valor_dict)
        
        
    return dados_prefeitura 
                
def cabecalho_dados():

    valor = {}   
    f_frame_name = "1_frame_dados_nf"
    
    dadinho_dados_nf = {}
    
    # 1. funçao basica de modelo 
    texto = executa_model_frame(model, secao, f_frame_name)    
    text_splited = texto_extraido(texto)
    keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']

    string_pesquisa = "Número da Nota:"
    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)         
    dadinho_dados_nf['numero_nota_fiscal'] = texto


    string_pesquisa = "Competência:"
    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
    dadinho_dados_nf['competencia'] = texto
    
    
    string_pesquisa = "ata e Hora da Emissão:"
    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
    dadinho_dados_nf['dt_hr_emissao'] = texto
    
    
    string_pesquisa = "Código Verificação:"
    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
    dadinho_dados_nf['codigo_verificacao'] = texto
    
    return dadinho_dados_nf

#4. Extrai prefeitura
def extract_prefeitura(model, father, values):
    
    tipo = "sframe_field"
    data_extrated_prefeitura = {}
    #print(tipo)

    filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model) & (frames_nf_v4_df['father'] == father) & (frames_nf_v4_df['type'] == tipo)]

    for index_sframe, row_sframe in filtered_frames_nf_v4_df.iterrows():
        
        label_value = row_sframe['label']
        
        #print("label_value", label_value)
        
        if label_value == "nome_prefeitura":
            reference_value = row_sframe['reference']
            for value in values:
                result = process_line(value, reference_value, label_value)
                if result:
                    data_extrated_prefeitura.update(result)
        elif label_value == "secretaria":
            reference_value = row_sframe['reference']
            for value in values:
                result = process_line(value, reference_value, label_value)
                if result:
                    data_extrated_prefeitura.update(result) 
        elif label_value == "tipo_nota_fiscal":
            reference_value = row_sframe['reference']  
            for value in values:
                result = process_line(value, reference_value, label_value)
                if result:
                    data_extrated_prefeitura.update(result)
                    
    return data_extrated_prefeitura



def cabecalho_prefeitura():
    valor_dict = {}
    dados_prefeitura = {}
    f_frame_name = "1_frame_prefeitura_nf"
    # 1. funçao basica de modelo 
    texto = executa_model_frame(model, secao, f_frame_name)
    text_splited = texto.split('\n')
    
    valor_dict = extract_prefeitura(model, f_frame_name, text_splited)
    if valor_dict:
        dados_prefeitura.update(valor_dict)
        
        
    return dados_prefeitura 
                
def cabecalho_dados():

    valor = {}   
    f_frame_name = "1_frame_dados_nf"
    
    dadinho_dados_nf = {}
    
    # 1. funçao basica de modelo 
    texto = executa_model_frame(model, secao, f_frame_name)    
    text_splited = texto_extraido(texto)
    keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']

    string_pesquisa = "Número da Nota:"
    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)         
    dadinho_dados_nf['numero_nota_fiscal'] = texto


    string_pesquisa = "Competência:"
    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
    dadinho_dados_nf['competencia'] = texto
    
    
    string_pesquisa = "ata e Hora da Emissão:"
    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
    dadinho_dados_nf['dt_hr_emissao'] = texto
    
    
    string_pesquisa = "Código Verificação:"
    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
    dadinho_dados_nf['codigo_verificacao'] = texto
    
    return dadinho_dados_nf  


In [ ]:
def processar_cabecalho(row, model_map, pdf_pesquisavel_map, original_file_name, file_path, frame_type):
    
    
    section = "1 - CABECALHO"
    f_frame_label = "1_frame_dados_nf"
    data_box_valores = {'secao': section}
    nf_data_cabecalho = {}
    
 
    message_erro = []
    
    model = model_map
    pdf_pesquisavel = pdf_pesquisavel_map
    
    if pdf_pesquisavel_map:
        f_tipo = 'frame'
        
    filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model) & (frames_nf_v4_df['type'] == frame_type) & (frames_nf_v4_df['label'] == f_frame_label)]
    
    if pdf_pesquisavel:
        print(' ')
        # pdf_document = fitz.open(file_path)
        # # Página do PDF
        # page_number = 0  # Defina o número da página que deseja analisar
        # page = pdf_document[page_number]
        # x0, y0, x1, y1 = (0, 0, 600, 110)
        # text = page.get_text("text", clip=(x0, y0, x1, y1))
        # nf_data_cabecalho = extract_fields_cabecalho(text)
        
        # pdf_document.close()
    else:
        image_2work, image_resized_name = convertResize(original_file_name, file_path, image_resized_path)

    for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
        frame_id = row_frame['id']
        label = row_frame['label']
        x0, y0, x1, y1 = (row_frame['x0_p'], row_frame['y0_p'], row_frame['x1_p'], row_frame['y1_p']) if pdf_pesquisavel else (row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1'])
        
        #print(f'doc {"pdf_pesquisavel" if pdf_pesquisavel else "raster_pdf"}: {label:>30} | id: {frame_id:>3} | x0: {x0:>6} y0: {y0:>6} x1: {x1:>6} y1: {y1:>6} ')
        
        if pdf_pesquisavel:
            # texto_extraido = page.get_text("text", clip=(x0, y0, x1, y1))
            pdf_document = fitz.open(file_path)
            # Página do PDF
            page_number = 0  # Defina o número da página que deseja analisar
            page = pdf_document[page_number]
            #x0, y0, x1, y1 = (0, 0, 600, 110)
            text = page.get_text("text", clip=(x0, y0, x1, y1))
            nf_data_cabecalho = extract_fields_cabecalho(text)
            #x0, y0, x1, y1 = (0, 0, 600, 110)
            #text = page.get_text("text", clip=(x0, y0, x1, y1))
            #data_servico = extract_fields_cabecalho()
            nf_data_cabecalho['original_file_name'] = original_file_name
            nf_data_cabecalho['pdf_pesquisavel'] = pdf_pesquisavel_map
            
            pdf_document.close()
            
            
            

        else:
            texto_extraido = extract_text_PIL(image_2work, (x0, y0, x1, y1))
            print(f'RASTER PDF file: {original_file_name:>50} | doc {"pdf_pesquisavel" if pdf_pesquisavel else "raster_pdf"}: {label:>30} | id: {frame_id:>3} | x0: {x0:>6} y0: {y0:>6} x1: {x1:>6} y1: {y1:>6}\n')
            #print(texto_extraido)
            
        try:
            text_splited = texto_extraido_cabecalho(texto_extraido)
            keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']

            string_pesquisa = "Número da Nota:"
            texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)         
            nro_nota_frame = texto
            #print(nro_nota_frame)
            nf_data_cabecalho['numero_nota_fiscal'] = nro_nota_frame
            
            string_pesquisa = "Competência:"
            texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            competencia_frame = texto
            nf_data_cabecalho['competencia'] = competencia_frame 
            #print(competencia_frame)
            
            string_pesquisa = "ata e Hora da Emissão:"
            texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            dt_hr_emissao_frame = texto
            nf_data_cabecalho['dt_hr_emissao'] = dt_hr_emissao_frame 
            
            string_pesquisa = "Código Verificação:"
            texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            codigo_ver_frame = texto
            nf_data_cabecalho['codigo_verificacao'] = codigo_ver_frame 
            
            
            
            #print(codigo_ver_frame)
            print(f'label: {label} | pdf_pesq.: {pdf_pesquisavel} | nro: {nro_nota_frame} | competencia: {competencia_frame} | dt_hr_emissao: {dt_hr_emissao_frame} | codigo_ver: {codigo_ver_frame} | file: {original_file_name}\n')
            print(text_splited)
            print()
            
            
        except Exception as e:
            # erros_cabecalho = {}
            msg = (f"doc: {original_file_name} | {e}")
            nro_nota_frame = " "
            competencia_frame = " "
            dt_hr_emissao_frame = " "
            codigo_ver_frame = " "
            message_erro.append(msg)
    
    # if pdf_pesquisavel:
    #     pdf_document.close()
        
    return nf_data_cabecalho






def cabecalho_prefeitura():
    valor_dict = {}
    dados_prefeitura = {}
    f_frame_name = "1_frame_prefeitura_nf"
    # 1. funçao basica de modelo 
    texto = executa_model_frame(model, secao, f_frame_name)
    text_splited = texto.split('\n')
    
    valor_dict = extract_prefeitura(model, f_frame_name, text_splited)
    if valor_dict:
        dados_prefeitura.update(valor_dict)
        
        
    return dados_prefeitura 
                
def cabecalho_dados():

    valor = {}   
    f_frame_name = "1_frame_dados_nf"
    
    dadinho_dados_nf = {}
    
    # 1. funçao basica de modelo 
    texto = executa_model_frame(model, secao, f_frame_name)    
    text_splited = texto_extraido(texto)
    keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']

    string_pesquisa = "Número da Nota:"
    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)         
    dadinho_dados_nf['numero_nota_fiscal'] = texto


    string_pesquisa = "Competência:"
    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
    dadinho_dados_nf['competencia'] = texto
    
    
    string_pesquisa = "ata e Hora da Emissão:"
    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
    dadinho_dados_nf['dt_hr_emissao'] = texto
    
    
    string_pesquisa = "Código Verificação:"
    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
    dadinho_dados_nf['codigo_verificacao'] = texto
    
    return dadinho_dados_nf   

In [ ]:
def extract_fields_cabecalho_pdf_pesquisavel(row, file_path):
    
    message_erro = [] 
    pdf_document = fitz.open(file_path)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]
    
    
    model_src = row['modelo']
    original_file_name = row['original_file_name']
    tipo = "frame"
    section = '1 - CABECALHO'
    label = '1_frame_prefeitura_nf'

    x0, y0, x1, y1 = pesquisa_frame_coordenadas_pdf_pesquisa(model_src, tipo, section, label)

    try:
        texto = page.get_text("text", clip=(x0, y0, x1, y1))
        
        text_splited = texto_extraido_nf(texto)
        keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']

        string_pesquisa = "Número da Nota:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)         
        nro_nota_frame = texto
        
        string_pesquisa = "Competência:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        competencia_frame = texto
        
        string_pesquisa = "ata e Hora da Emissão:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        dt_hr_emissao_frame = texto
        
        string_pesquisa = "Código Verificação:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        codigo_verificacao_frame = texto
    except Exception as e:
        msg = (f"doc: {original_file_name} | {e}")
        nro_nota_frame = " "
        competencia_frame = " "
        dt_hr_emissao_frame = " "
        codigo_verificacao_frame = " "
        message_erro.append(msg)
            
    pdf_document.close()

In [ ]:
def extract_fields_cabecalho_raster_pdf(row, file_path, image_2work): 
  
    message_erro = []
  
  
    model_src = row['modelo']
    original_file_name = row['original_file_name'] 
    
    model = row['modelo']   
    secao = "1 - CABECALHO"
    section = "1 - CABECALHO"
    f_frame_name = "1_frame_dados_nf"
    f_tipo = 'frame'
        
    raw_texto_pil = executa_model_frame(model, secao, f_frame_name, f_tipo, image_2work) # executa_model_frame
    
    
    try:
        text_splited = texto_extraido_cabecalho(raw_texto_pil)
        keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']

        string_pesquisa = "Número da Nota:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)         
        nro_nota_frame = texto
        
        string_pesquisa = "Competência:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        competencia_frame = texto
        
        string_pesquisa = "ata e Hora da Emissão:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        dt_hr_emissao_frame = texto
        
        string_pesquisa = "Código Verificação:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        codigo_verificacao_frame = texto
        
    except Exception as e:
        # erros_cabecalho = {}
        msg = (f"doc: {original_file_name} | {e}")
        nro_nota_frame = " "
        competencia_frame = " "
        dt_hr_emissao_frame = " "
        codigo_verificacao_frame = " "
        message_erro.append(msg)
        
        
    return nro_nota_frame, competencia_frame, dt_hr_emissao_frame, codigo_verificacao_frame, message_erro, section, raw_texto_pil 

def extract_fields_cabecalho_pdf_pesquisavel(row, file_path):
    
    message_erro = [] 
    pdf_document = fitz.open(file_path)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]
    
    
    model_src = row['modelo']
    original_file_name = row['original_file_name']
    tipo = "frame"
    section = '1 - CABECALHO'
    label = '1_frame_prefeitura_nf'

    x0, y0, x1, y1 = pesquisa_frame_coordenadas_pdf_pesquisa(model_src, tipo, section, label)

    try:
        texto = page.get_text("text", clip=(x0, y0, x1, y1))
        
        text_splited = texto_extraido_nf(texto)
        keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']

        string_pesquisa = "Número da Nota:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)         
        nro_nota_frame = texto
        
        string_pesquisa = "Competência:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        competencia_frame = texto
        
        string_pesquisa = "ata e Hora da Emissão:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        dt_hr_emissao_frame = texto
        
        string_pesquisa = "Código Verificação:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        codigo_verificacao_frame = texto
    except Exception as e:
        msg = (f"doc: {original_file_name} | {e}")
        nro_nota_frame = " "
        competencia_frame = " "
        dt_hr_emissao_frame = " "
        codigo_verificacao_frame = " "
        message_erro.append(msg)
            
    pdf_document.close()
    #print(row)
    return nro_nota_frame, competencia_frame, dt_hr_emissao_frame, codigo_verificacao_frame, message_erro, section

In [ ]:
def extract_fields_cabecalho_raster_pdf(row, file_path, image_2work): 
  
    message_erro = []
  
  
    model_src = row['modelo']
    original_file_name = row['original_file_name'] 
    
    model = row['modelo']   
    secao = "1 - CABECALHO"
    section = "1 - CABECALHO"
    f_frame_name = "1_frame_dados_nf"
    f_tipo = 'frame'
        
    raw_texto_pil = executa_model_frame(model, secao, f_frame_name, f_tipo, image_2work) # executa_model_frame
    
    
    try:
        text_splited = texto_extraido_cabecalho(raw_texto_pil)
        keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']

        string_pesquisa = "Número da Nota:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)         
        nro_nota_frame = texto
        
        string_pesquisa = "Competência:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        competencia_frame = texto
        
        string_pesquisa = "ata e Hora da Emissão:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        dt_hr_emissao_frame = texto
        
        string_pesquisa = "Código Verificação:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        codigo_verificacao_frame = texto
        
    except Exception as e:
        # erros_cabecalho = {}
        msg = (f"doc: {original_file_name} | {e}")
        nro_nota_frame = " "
        competencia_frame = " "
        dt_hr_emissao_frame = " "
        codigo_verificacao_frame = " "
        message_erro.append(msg)
        
        
    return nro_nota_frame, competencia_frame, dt_hr_emissao_frame, codigo_verificacao_frame, message_erro, section, raw_texto_pil 

In [ ]:
def extract_fields_cabecalho_pdf_pesquisavel(row, file_path):
    
    message_erro = [] 
    pdf_document = fitz.open(file_path)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]
    
    
    model_src = row['modelo']
    original_file_name = row['original_file_name']
    tipo = "frame"
    section = '1 - CABECALHO'
    label = '1_frame_prefeitura_nf'

    x0, y0, x1, y1 = pesquisa_frame_coordenadas_pdf_pesquisa(model_src, tipo, section, label)

    try:
        texto = page.get_text("text", clip=(x0, y0, x1, y1))
        
        text_splited = texto_extraido_nf(texto)
        keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']

        string_pesquisa = "Número da Nota:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)         
        nro_nota_frame = texto
        
        string_pesquisa = "Competência:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        competencia_frame = texto
        
        string_pesquisa = "ata e Hora da Emissão:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        dt_hr_emissao_frame = texto
        
        string_pesquisa = "Código Verificação:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        codigo_verificacao_frame = texto
    except Exception as e:
        msg = (f"doc: {original_file_name} | {e}")
        nro_nota_frame = " "
        competencia_frame = " "
        dt_hr_emissao_frame = " "
        codigo_verificacao_frame = " "
        message_erro.append(msg)
            
    pdf_document.close()
    #print(row)
    return nro_nota_frame, competencia_frame, dt_hr_emissao_frame, codigo_verificacao_frame, message_erro, section

In [ ]:
# 1. funçao basica de modelo 
def pesquisa_model_frame(model_src, tipo):

    data_dados_model = {}
    dados_modelo = []
    
    filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model_src) & (frames_nf_v4_df['type'] == tipo)]
    
    for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
        
        frame_model = row_frame['model']
        # frame_prefeitura = row_frame['prefeitura']
        # frame_label = row_frame['label']
        # frame_section_json = row_frame['section_json']
        # frame_cnpj = row_frame['cnpj']
        
        # if frame_model:
        #     modelo_frame = True
        # else:    
        #     modelo_frame = False

                  
  
    return frame_model

In [ ]:
# 1. XXX funçao basica de modelo 
def pesquisa_frame_coordenadas_pdf_pesquisa(model_src, tipo, section, label):

    tipo = tipo
    data_dados_model = {}
    dados_modelo = []
    
    filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model_src) & (frames_nf_v4_df['type'] == tipo) & (frames_nf_v4_df['section_json'] == section) & (frames_nf_v4_df['label'] == label)]
    
    for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
        
        frame_model = row_frame['model']
        frame_label = row_frame['label']
 
        # data for PDF Pesquisavel
        x0, y0, x1, y1 = row_frame['x0_p'], row_frame['y0_p'], row_frame['x1_p'], row_frame['y1_p']
 
    return x0, y0, x1, y1

In [ ]:
# 1. XXX Avaliar Funçao basica de modelo 
def pesquisa_model_frame_coordenadas(model_src, tipo, tipo_processo_pdf):

    data_dados_model = {}
    dados_modelo = []
    
    filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model_src) & (frames_nf_v4_df['type'] == tipo)]
    
    for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
        
        frame_model = row_frame['model']
        frame_prefeitura = row_frame['prefeitura']
        frame_label = row_frame['label']
        frame_section_json = row_frame['section_json']
        frame_cnpj = row_frame['cnpj']
        
        if frame_model:
            modelo = True
        
        if tipo_processo_pdf == 'pdf_pesquisavel':
            try:
                # data for PDF Pesquisavel
                x0, y0, x1, y1 = row_frame['x0_p'], row_frame['y0_p'], row_frame['x1_p'], row_frame['y1_p']
                
                data_dados_model = {
                    "prefeitura": frame_prefeitura,
                    "model": frame_model,
                    "label": frame_label,
                    "section_json": frame_section_json,
                    "cnpj": frame_cnpj,
                    "x0": x0,
                    "y0": y0,
                    "x1": x1,
                    "y1": y1,
                }
                dados_modelo.append(data_dados_model)   
        
            except Exception as e:
                print(f"Erro ao carregar coordenadas do frame: {e}")
                x0, y0, x1, y1 = 0, 0, 0, 0 
                

        elif tipo_processo_pdf == 'raster_pdf':
            try:
                # data for Raster_PDF
                x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
                
                data_dados_model = {
                    "prefeitura": frame_prefeitura,
                    "model": frame_model,
                    "label": frame_label,
                    "section_json": frame_section_json,
                    "cnpj": frame_cnpj,
                    "x0": x0,
                    "y0": y0,
                    "x1": x1,
                    "y1": y1,
                }
                dados_modelo.append(data_dados_model)
            except Exception as e:
                print(f"Erro ao carregar coordenadas do frame: {e}")
                x0, y0, x1, y1 = 0, 0, 0, 0 
                  
  
    return dados_modelo, modelo

In [ ]:
dict_raw = {}
i = 0
for  doc in raw_document_list:
    dict_raw = raw_document_list[i]
    print(i)
    print(dict_raw['pdf_pesquisavel'])
    print(dict_raw['original_file_name'])
    print(dict_raw['texto_extraido'])
    #print(f'\nraw_document_list[i]: {raw_document_list[i]}\n\n')
    i += 1

In [ ]:
# Pesquisando os dados para o processo de extracao - Raster

#f_type = 'frame'
f_type = 'field_box'

filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['de_para_pm'] == de_para_pm) & (frames_nf_v4_df['model'] == model2search) & (frames_nf_v4_df['type'] == f_type)]

for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
    seq = row_frame['seq']
    de_para = row_frame['de_para_pm']
    model = row_frame['model']  
    print(f'{seq:>4} {de_para:>8} | {model:>10} | {row_frame["type"]:>15} | {row_frame["label"]:>35} | {row_frame["section_json"]:>35} | {row_frame["x0"]:>7} | {row_frame["y0"]:>7} | {row_frame["x1"]:>7} | {row_frame["y1"]:>7}')

In [ ]:
# Pesquisando os dados para o processo de extracao - PDF Pesquisavel

f_type = 'frame'

filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['de_para_pm'] == de_para_pm) & (frames_nf_v4_df['model'] == model2search) & (frames_nf_v4_df['type'] == f_type)]


for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
    seq = row_frame['seq']
    de_para = row_frame['de_para_pm']
    model = row_frame['model']  
    print(f'{seq:>4} {de_para:>8} | {model:>10} | {row_frame["type"]:>15} | {row_frame["label"]:>35} | {row_frame["section_json"]:>35} | {row_frame["x0_p"]:>7} | {row_frame["y0_p"]:>8} | {row_frame["x1_p"]:>8} | {row_frame["y1_p"]:>8}')

In [ ]:
filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model2search) & (frames_nf_v4_df['type'] == tipo)]

In [ ]:
import pandas as pd

# Criando um DataFrame de exemplo
data = {'A': [1, 2], 'B': [3, 4], 'C': [5, 6]}
df = pd.DataFrame(data)

# Criando uma lista para armazenar os dicionários que representam cada linha
lista_dicts = []

# Iterando sobre as linhas do DataFrame e adicionando cada linha à lista
for index, row in df.iterrows():
    # Convertendo a linha a um dicionário
    row_dict = row.to_dict()
    
    # Adicionando novas colunas ao dicionário
    row_dict['D'] = row['A'] * row['B']
    row_dict['E'] = row['B'] + row['C']
    
    # Adicionando o dicionário à lista
    lista_dicts.append(row_dict)

# Criando um novo DataFrame a partir da lista de dicionários
novo_df = pd.DataFrame(lista_dicts)

# Exibindo o novo DataFrame
print(novo_df)

In [ ]:
# 3. TOMADOR DE SERVIÇO
def extract_fields_tomador(text):
    nf_data_tomador = {}
    
    
    nf_data_tomador['secao'] = "3. TOMADOR DE SERVIÇO"
    
    
    # Extrair CPF/CNPJ com máscara 1
    if "CPF/CNPJ:" in text:
        cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', text)
        if cpf_cnpj_formatado_match:
                        nf_data_tomador['cpf_cnpj_com_mascara'] = cpf_cnpj_formatado_match.group(1)
                        nf_data_tomador['cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))

        
    # Extrair RG    
    rg_match = re.search(r'RG:\s+(.+)', text)   
    if rg_match:
        rg_str = rg_match.group(1)
        if rg_str == 'Telefone:':
            nf_data_tomador['rg'] = "NONE"  # Valor padrão quando não há correspondência
        else:    
            nf_data_tomador['rg'] = rg_match.group(1)  
 
        
    # Extrair Telefone
    telefone_match = re.search(r'Telefone:\s+(.+)', text)
    if telefone_match:
        telefone_str = telefone_match.group(1)
        if telefone_str == 'Inscrição Estadual:':
            nf_data_tomador['telefone'] = "NONE"  # Valor padrão quando não há correspondência
        else:    
            nf_data_tomador['telefone'] = telefone_match.group(1)
     

    # Extrair Inscrição Municipal
    inscricao_municipal_match = re.search(r'Inscrição Municipal:\s+(.+)', text)
    if inscricao_municipal_match:
        nf_data_tomador['inscricao_municipal'] = inscricao_municipal_match.group(1)
                
                
                
    # Extrair Inscrição Estadual
    inscricao_estadual_match = re.search(r'Inscrição Estadual:\s+(.+)', text)
    if inscricao_estadual_match:
        inscricao_estadual_str = inscricao_estadual_match.group(1)
        if inscricao_estadual_str == 'Nome/Razão Social:':
            nf_data_tomador['inscricao_estadual'] = "NONE"
        else:    
            nf_data_tomador['inscricao_estadual'] = inscricao_estadual_match.group(1)   
                
    
    # Nome/Razão Social:
    razao_social_match = re.search(r'Nome/Razão Social:\s+(.+)', text)
    if razao_social_match:
        nf_data_tomador['razao_social'] = razao_social_match.group(1)                                                
                
    # Endereço:
    endereco_match = re.search(r'Endereço:\s+(.+)', text)
    if endereco_match:
        nf_data_tomador['endereco'] = endereco_match.group(1) 
    
    # E-mail:
    email_match = re.search(r'E-mail:\s+(.+)', text)
    if email_match:
        nf_data_tomador['email'] = email_match.group(1) 
    else:
        nf_data_tomador['email'] = "NONE"  # Valor padrão quando não há correspondência    

    return nf_data_tomador

In [ ]:

    # Extrair EXIGIBILIDADE ISS:
    try:
        exigibilidade_iss_match = re.search(r'EXIGIBILIDADE ISS\s+(.+)', text)
        if exigibilidade_iss_match:
            exigibilidade_iss_value = exigibilidade_iss_match.group(1).strip()
            nf_data_outras_informacoes['exigibilidade_iss'] = exigibilidade_iss_value
    except Exception as e:
        msg = (f"doc: {map_original_file_name} | {e}")
        exigibilidade_iss_value = ' '

    try:
        # Extrair REGIME TRIBUTAÇÃO:
        regime_tributacao_match = re.search(r'REGIME TRIBUTAÇÃO\s+(.+)', text)
        if regime_tributacao_match:
            regime_tributacao_value = regime_tributacao_match.group(1).strip()
            nf_data_outras_informacoes['regime_tributacao'] = regime_tributacao_value
    except Exception as e:
        msg = (f"doc: {map_original_file_name} | {e}")
        regime_tributacao_value = ' '    
    
    try:
        # Extrair SIMPLES NACIONAL:
        simples_nacional_match = re.search(r'SIMPLES NACIONAL\s+(.+)', text)
        if simples_nacional_match:
            simples_nacional_value = simples_nacional_match.group(1).strip()
            nf_data_outras_informacoes['simples_nacional'] = simples_nacional_value  
    except Exception as e:
        msg = (f"doc: {map_original_file_name} | {e}")
        simples_nacional_value = ' '     
        
      
    try:
        # Extrair ISSQN RETIDO:
        local_prestacao_servico_match = re.search(r'ISSQN RETIDO\s+(.+)', text)
        if local_prestacao_servico_match:
            local_prestacao_servico_value = local_prestacao_servico_match.group(1).strip()
            nf_data_outras_informacoes['issqn_retido'] = local_prestacao_servico_value      
    except Exception as e:
        msg = (f"doc: {map_original_file_name} | {e}")
        exigibilidade_iss_value = ' ' 
    
    
    try:
        # Extrair LOCAL PRESTAÇÃO SERVIÇO:
        local_prestacao_servico_match = re.search(r'LOCAL\. PRESTAÇÃO\s+SERVIÇO\s+(.+)', text)
        if local_prestacao_servico_match:
            local_prestacao_servico_value = local_prestacao_servico_match.group(1).strip()
            nf_data_outras_informacoes['local_prestacao_servico'] = local_prestacao_servico_value
    except Exception as e:
        msg = (f"doc: {map_original_file_name} | {e}")
        local_prestacao_servico_value = ' ' 
    
    
    try:
        # Extrair LOCAL INCIDÊNCIA:
        local_incidencia_match = re.search(r'LOCAL INCIDÊNCIA\s+(.+)', text)
        if local_incidencia_match:
            local_incidencia_value = local_incidencia_match.group(1).strip()
            nf_data_outras_informacoes['local_incidencia'] = local_incidencia_value
    except Exception as e:
        msg = (f"doc: {map_original_file_name} | {e}")
        local_incidencia_value = ' '    
    
  
    
    pdf_document.close()
    return nf_data_outras_informacoes
 

In [ ]:
# 2. PRESTADOR DE SERVIÇO
def extract_fields_prestador(text): # Função para extrair campos e valores dentro de um retângulo
    nf_data_prestador = {}
    
    nf_data_prestador['secao'] = "2. PRESTADOR DE SERVIÇO"
    
    
    # Extrair CPF/CNPJ com máscara 1
    if "CPF/CNPJ:" in text:
        cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', text)
        if cpf_cnpj_formatado_match:
                        nf_data_prestador['cpf_cnpj_com_mascara'] = cpf_cnpj_formatado_match.group(1)
                        nf_data_prestador['cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))

    # Extrair Inscrição Municipal
    inscricao_municipal_match = re.search(r'Inscrição Municipal:\s+(.+)', text)
    if inscricao_municipal_match:
        nf_data_prestador['inscricao_municipal'] = inscricao_municipal_match.group(1)
        
               
    # Extrair Inscrição Estadual
    #if "Inscrição Estadual:" in text:
    
    # Extrair Inscrição Estadual
    inscricao_estadual_match = re.search(r'Inscrição Estadual:\s+(.+)', text)
    if inscricao_estadual_match:
        inscricao_estadual_str = inscricao_estadual_match.group(1)
        if inscricao_estadual_str == 'Nome/Razão Social:':
            nf_data_prestador['inscricao_estadual'] = "NONE"
        else:    
            nf_data_prestador['inscricao_estadual'] = inscricao_estadual_match.group(1)       
        
                
    

    # Extrair Telefone
    #telefone_match = re.search(r'Telefone:\s+([0-9.\s-])', text)
    telefone_match = re.search(r'Telefone:\s+([0-9.\s-]+)', text)
    if telefone_match: 
        telefone_str = telefone_match.group(1)
        # Remover quebras de linha
        telefone_str = telefone_str.replace('.', '')
        telefone_str = telefone_str.replace('\n', '')
                
        nf_data_prestador['telefone'] = telefone_str
    else:
        nf_data_prestador['telefone'] = "NONE"

         
                
    # Nome/Razão Social:
    razao_social_match = re.search(r'Nome/Razão Social:\s+(.+)', text)
    if razao_social_match:
        nf_data_prestador['razao_social'] = razao_social_match.group(1)  
                
    # Nome de Fantasia:
    nome_fantasia_match = re.search(r'Nome de Fantasia:\s+(.+)', text)
    if nome_fantasia_match:
        nf_data_prestador['nome_fantasia'] = nome_fantasia_match.group(1)                                    
                
            
    # Endereço:
    endereco_match = re.search(r'Endereço:\s+(.+)', text)
    if endereco_match:
        nf_data_prestador['endereco'] = endereco_match.group(1) 
    
    # E-mail:
    email_match = re.search(r'E-mail:\s+(.+)', text)
    if email_match:
        nf_data_prestador['email'] = email_match.group(1)  
    else:
        nf_data_prestador['email'] = "NONE"  # Valor padrão quando não há correspondência
   
        

    return nf_data_prestador

## VERIFICAR
def extract_fields_cnpj(texto_extraido, original_file_name): 
  
    message_erro = []
    
    data_nf_cnpj_prestador = {}
      
    #print(f'2 extract_fields_cnpj:  {texto_extraido}')
    
    # Extrair CPF/CNPJ com máscara 1
    if "CPF/CNPJ:" in texto_extraido:
        cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', texto_extraido)
        if cpf_cnpj_formatado_match:
            data_nf_cnpj_prestador['cpf_cnpj_com_mascara'] = cpf_cnpj_formatado_match.group(1)
            data_nf_cnpj_prestador['cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))
    else:
        data_nf_cnpj_prestador['cpf_cnpj_com_mascara'] = None
        data_nf_cnpj_prestador['cpf_cnpj_sem_mascara'] = None            
            
            
    telefone_str = None

    #telefone_match = re.search(r'Telefone:\s+([0-9.\s-])', text)
    telefone_match = re.search(r'Telefone:\s+([0-9.\s-]+)', texto_extraido)
    if telefone_match: 
        telefone_str = telefone_match.group(1)
        # Remover quebras de linha
        telefone_str = telefone_str.replace('.', '')
        telefone_str = telefone_str.replace('\n', '')
                
        data_nf_cnpj_prestador['telefone'] = telefone_str
    else:
        data_nf_cnpj_prestador['telefone'] = None 
    
    #print(data_nf_cnpj_prestador)    

             
    return data_nf_cnpj_prestador

In [ ]:
# VERRIFICAR (under cosntruction)
def processar_servicos_pdf_pesquisavel_2(row, pdf_pesquisavel_map, model_map, original_file_name, file_path):
    
    nf_data_servico = {}
    
    process = ['4_frame_descricao_totais']
    
    section = "4. DESCRIMINACAO DOS SERVIÇOS"
    pdf_pesquisavel_map = True
        
    tipo= "frame"
    message_erro = []
    nf_dados_prestador = {}
    model = row['model']
    print(model)

    for father in process_prestador:
        label = father
        if label == "4_frame_descricao_totais": 
            coordinates = get_coordinates_filter(model_map, tipo, label, section)
            x0, y0, x1, y1 = coordinates[0]
            extract_text = extract_text_PIL(image_2work, (x0, y0, x1, y1))
            #text_splited = texto_extraido_nf(extract_text)
            if "CPF/CNPJ:" in extract_text:
                cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', extract_text)
                if cpf_cnpj_formatado_match:
                    texto = cpf_cnpj_formatado_match.group(1)
                
                    nf_dados_prestador['cpf_cnpj_com_mascara'] = texto
                    nf_dados_prestador['cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))
            else:
                nf_dados_prestador['cpf_cnpj_com_mascara'] = 'None'
                nf_dados_prestador['cpf_cnpj_sem_mascara'] = 'None' 

In [ ]:
# 7. VALORES E IMPOSTOS
def extract_fields_impostos(text):
    nf_data_valores = {}
    #nf_data_valores['secao'] = "7. VALORES E IMPOSTOS"
    
    # Extrair VALOR SERVIÇOS:
    valor_servicos_match = re.search(r'VALOR SERVIÇOS:\s+(.+)', text)
    if valor_servicos_match:
        valor_servicos_str = valor_servicos_match.group(1)
        valor_servicos_sem_formato = re.search(r'R\$\s*([\d.,]+)', valor_servicos_str)
        if valor_servicos_sem_formato:
            valor_servicos_sem_formatacao = valor_servicos_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['valor_servicos'] = float(valor_servicos_sem_formatacao)
        else:
            nf_data_valores['valor_servicos'] = 0.0  # Valor não encontrado ou não está no formato esperado
  
  
    # Extrair VALOR DEDUÇÃO:
    valor_deducao_match = re.search(r'DEDUÇÃO:\s+(.+)', text)
    if valor_deducao_match:
        valor_deducao_str = valor_deducao_match.group(1)
        valor_deducao_sem_formato = re.search(r'R\$\s*([\d.,]+)', valor_deducao_str)
        if valor_deducao_sem_formato:
            valor_deducao_sem_formato = valor_deducao_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['valor_deducao'] = float(valor_deducao_sem_formato)
        else:
            nf_data_valores['valor_deducao'] = 0.0  # Valor não encontrado ou não está no formato esperado
        
        
    # Extrair DESC. INCOND:
    valor_desc_match = re.search(r'DESC. INCOND:\s+(.+)', text)
    if valor_desc_match:
        valor_desc_str = valor_desc_match.group(1)
        valor_desc_sem_formato = re.search(r'R\$\s*([\d.,]+)', valor_desc_str)
        if valor_desc_sem_formato:
            valor_desc_sem_formato = valor_desc_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['desc_incond'] = float(valor_desc_sem_formato)
        else:
            nf_data_valores['desc_incond'] = 0.0  # Valor não encontrado ou não está no formato esperado        
        

    # Extrair BASE DE CÁLCULO:
    valor_calculo_match = re.search(r'CÁLCULO:\s+(.+)', text)
    if valor_calculo_match:
        valor_calculo_str = valor_calculo_match.group(1)
        valor_calculo_sem_formato = re.search(r'R\$\s*([\d.,]+)', valor_calculo_str)
        if valor_calculo_sem_formato:
            valor_calculo_sem_formato = valor_calculo_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['base_calculo'] = float(valor_calculo_sem_formato)
        else:
            nf_data_valores['base_calculo'] = 0.0  # Valor não encontrado ou não está no formato esperado    



    # Extrair ALÍQUOTA:
    valor_aliquota_match = re.search(r'ALÍQUOTA:\s+(.+)', text)
    if valor_aliquota_match:
        valor_aliquota_str = valor_aliquota_match.group(1)
        valor_aliquota_sem_formato = re.search(r'([\d.,]+)%', valor_aliquota_str)  # Ajuste aqui
        if valor_aliquota_sem_formato:
            valor_aliquota_sem_formato = valor_aliquota_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['aliquota'] = float(valor_aliquota_sem_formato)
        else:
            nf_data_valores['aliquota'] = 0.0  # Valor não encontrado ou não está no formato esperado


    # Extrair VALOR ISS:
    valor_iss_match = re.search(r'VALOR ISS:\s+(.+)', text)
    if valor_iss_match:
        valor_iss_str = valor_iss_match.group(1)
        valor_iss_sem_formato = re.search(r'R\$\s*([\d.,]+)', valor_iss_str)
        if valor_iss_sem_formato:
            valor_iss_sem_formato = valor_iss_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['valor_iss'] = float(valor_iss_sem_formato)
        else:
            nf_data_valores['valor_iss'] = 0.0  # Valor não encontrado ou não está no formato esperado 

    # Extrair VALOR ISS RETIDO:
    valor_iss_retido_match = re.search(r'RETIDO:\s+(.+)', text)
    if valor_iss_match:
        valor_iss_retido_str = valor_iss_retido_match.group(1)
        valor_iss_retido_sem_formato = re.search(r'R\$\s*([\d.,]+)', valor_iss_retido_str)
        if valor_iss_retido_sem_formato:
            valor_iss_retido_sem_formato = valor_iss_retido_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['valor_iss_retido'] = float(valor_iss_retido_sem_formato)
        else:
            nf_data_valores['valor_iss_retido'] = 0.0  # Valor não encontrado ou não está no formato esperado 

    # Extrair VALOR DESC. COND:
    valor_desc_cond_match = re.search(r'DESC. COND:\s+(.+)', text)
    if valor_desc_cond_match:
        valor_desc_cond_str = valor_desc_cond_match.group(1)
        valor_desc_cond_sem_formato = re.search(r'R\$\s*([\d.,]+)', valor_desc_cond_str)
        if valor_desc_cond_sem_formato:
            valor_desc_cond_sem_formato = valor_desc_cond_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['desc_cond'] = float(valor_desc_cond_sem_formato)
        else:
            nf_data_valores['desc_cond'] = 0.0  # Valor não encontrado ou não está no formato esperado
    
    # Extrair VALOR PIS:
    valor_pis_match = re.search(r'VALOR PIS:\s+(.+)', text)
    if valor_pis_match:
        valor_pis_str = valor_pis_match.group(1)
        valor_pis_sem_formato = re.search(r'R\$\s*([\d.,]+)', valor_pis_str)
        if valor_pis_sem_formato:
            valor_pis_sem_formato = valor_pis_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['valor_pis'] = float(valor_pis_sem_formato)
        else:
            nf_data_valores['valor_pis'] = 0.0  # Valor não encontrado ou não está no formato esperado
    
    # Extrair VALOR COFINS:
    valor_cofins_match = re.search(r'VALOR COFINS:\s+(.+)', text)
    if valor_cofins_match:
        valor_cofins_str = valor_cofins_match.group(1)
        valor_cofins_sem_formato = re.search(r'R\$\s*([\d.,]+)', valor_cofins_str)
        if valor_cofins_sem_formato:
            valor_cofins_sem_formato = valor_cofins_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['valor_cofins'] = float(valor_cofins_sem_formato)
        else:
            nf_data_valores['valor_cofins'] = 0.0  # Valor não encontrado ou não está no formato esperado
            
    # Extrair VALOR IR:
    valor_ir_match = re.search(r'VALOR IR:\s+(.+)', text)
    if valor_ir_match:
        valor_ir_str = valor_ir_match.group(1)
        valor_ir_sem_formato = re.search(r'R\$\s*([\d.,]+)', valor_ir_str)
        if valor_ir_sem_formato:
            valor_ir_sem_formato = valor_ir_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['valor_ir'] = float(valor_ir_sem_formato)
        else:
            nf_data_valores['valor_ir'] = 0.0  # Valor não encontrado ou não está no formato esperado
            
    # Extrair VALOR INSS:
    valor_inss_match = re.search(r'VALOR INSS:\s+(.+)', text)
    if valor_inss_match:
        valor_inss_str = valor_inss_match.group(1)
        valor_inss_sem_formato = re.search(r'R\$\s*([\d.,]+)', valor_inss_str)
        if valor_inss_sem_formato:
            valor_inss_sem_formato = valor_inss_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['valor_inss'] = float(valor_inss_sem_formato)
        else:
            nf_data_valores['valor_inss'] = 0.0  # Valor não encontrado ou não está no formato esperado
            
    # Extrair VALOR CSLL:
    valor_csll_match = re.search(r'VALOR CSLL:\s+(.+)', text)
    if valor_csll_match:
        valor_csll_str = valor_csll_match.group(1)
        valor_csll_sem_formato = re.search(r'R\$\s*([\d.,]+)', valor_csll_str)
        if valor_csll_sem_formato:
            valor_csll_sem_formato = valor_csll_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['valor_csll'] = float(valor_csll_sem_formato)
        else:
            nf_data_valores['valor_csll'] = 0.0  # Valor não encontrado ou não está no formato esperado
    
    # Extrair OUTRAS RETENÇÕES:
    outras_retencoes_match = re.search(r'OUTRAS RETENÇÕES:\s+(.+)', text)
    if outras_retencoes_match:
        outras_retencoes_str = outras_retencoes_match.group(1)
        outras_retencoes_sem_formato = re.search(r'R\$\s*([\d.,]+)', outras_retencoes_str)
        if outras_retencoes_sem_formato:
            outras_retencoes_sem_formato = outras_retencoes_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['outras_retencoes'] = float(outras_retencoes_sem_formato)
        else:
            nf_data_valores['outras_retencoes'] = 0.0  # Valor não encontrado ou não está no formato esperado
    
    
    # Extrair VALOR LÍQUIDO:
    valor_liquido_match = re.search(r'VALOR LÍQUIDO:\s+(.+)', text)
    if valor_liquido_match:
        valor_liquido_str = valor_liquido_match.group(1)
        valor_liquido_sem_formato = re.search(r'R\$\s*([\d.,]+)', valor_liquido_str)
        if valor_liquido_sem_formato:
            valor_liquido_sem_formato = valor_liquido_sem_formato.group(1).replace('.', '').replace(',', '.').strip()
            nf_data_valores['valor_liquido'] = float(valor_liquido_sem_formato)
        else:
            nf_data_valores['valor_liquido'] = 0.0  # Valor não encontrado ou não está no formato esperado
            
    #print(nf_data_valores)        
        

    return nf_data_valores

In [ ]:
# 1. CABECALHO
def extract_fields_cabecalho(text):
    nf_data_cabecalho = {}
    # Extrair Nome da Prefeitura
    nome_prefeitura_match = re.search(r'PREFEITURA (.+)', text)
    if nome_prefeitura_match:
        nome_prefeitura = "PREFEITURA " + nome_prefeitura_match.group(1)
        nf_data_cabecalho['nome_prefeitura'] = nome_prefeitura
    else:
        nf_data_cabecalho['nome_prefeitura'] = None    

    # Extrair Tipo de NF
    tipo_nf_match = re.search(r'NOTA FISCAL (.+)', text)
    if tipo_nf_match:
        tipo_nf = "NOTA FISCAL " + tipo_nf_match.group(1)
        nf_data_cabecalho['tipo_nota_fiscal'] = tipo_nf
    else:
        nf_data_cabecalho['tipo_nota_fiscal'] = None    
    
    # Extrair Número da Nota
    numero_nota_match = re.search(r'Número da Nota:\s+(\d+)', text)
    if numero_nota_match:
        nr_nro_nf = numero_nota_match.group(1)
        nf_data_cabecalho['numero_nota_fiscal'] = numero_nota_match.group(1)
    else:
        nf_data_cabecalho['numero_nota_fiscal'] = None
        nf_data_erros['cabecalho'] = "numero_nota"
        nf_lista_erros.append(nf_data_erros)        

    # Extrair Competência
    competencia_match = re.search(r'Competência:\s+(.+)', text)
    if competencia_match:
        nf_data_cabecalho['competencia'] = competencia_match.group(1)
    else:
        nf_data_cabecalho['competencia'] = None
        nf_data_erros['cabecalho'] = "competencia"
        nf_lista_erros.append(nf_data_erros)
            

    # Extrair Data e Hora de Emissão
    data_emissao_match = re.search(r'Data e Hora da Emissão:\s+(.+)', text)
    if data_emissao_match:
        nf_data_cabecalho['dt_hr_emissao'] = data_emissao_match.group(1)
    else:
        nf_data_cabecalho['dt_hr_emissao'] = None    
        
    # Extrair Data e Hora de Emissão
    codigo_verificacao_match = re.search(r'Código Verificação:\s+(.+)', text)
    if codigo_verificacao_match:
        nf_data_cabecalho['codigo_verificacao'] = codigo_verificacao_match.group(1)
    else:
        nf_data_cabecalho['codigo_verificacao'] = None
        nf_data_erros['cabecalho'] = "codigo_verificacao"
        nf_lista_erros.append(nf_data_erros)        

    return nf_data_cabecalho


In [ ]:
# 2. PRESTADOR DE SERVIÇO
def extrai_prestador_PDF_P(row, pdf_pesquisavel_map, de_para_pm, model_map, f_0, f_1, original_file_name, file_path):
    
    nf_data_prestador = {}
    section = "2. PRESTADOR DE SERVIÇO"
    pdf_pesquisavel_map = True
    
    label = "2_frame_cnpj_prestador"

    pdf_document = fitz.open(file_path)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]    
    tipo = "frame"

    coordinates = get_coordinates_filter(pdf_pesquisavel_map=pdf_pesquisavel_map, model=model_map, tipo=tipo, label=label, section=section)
    x0, y0, x1, y1 = coordinates[0]
    #print(label)
    #print(x0,x1,y0,y1)
    y0 = y0 * f_0
    y1 = y1 * f_1
    text = page.get_text("text", clip=(x0, y0, x1, y1))
    nf_data_prestador = novaextra.extract_fields_prestador(text)

           

    pdf_document.close()

    return nf_data_prestador 

In [ ]:
# 5. VALOR TOTAL
def processar_valor_total_PDF_P(row, pdf_pesquisavel_map, model_map, original_file_name, file_path):
    
    nf_data_valor_total = {}
    
    process = ['4_frame_valor_total']
    
    section = "5. VALOR TOTAL"
    pdf_pesquisavel_map = True
        
    pdf_document = fitz.open(file_path)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]    
        
    tipo= "frame"
    message_erro = []
    nf_dados_prestador = {}
    # model = row['model']
    for father in process:
        label = father
        if label == "4_frame_valor_total": 
            print(model_map)
            coordinates = get_coordinates_filter(pdf_pesquisavel_map=pdf_pesquisavel_map, model=model_map, tipo=tipo, label=label, section=section)
            x0, y0, x1, y1 = coordinates[0]
            text = page.get_text("text", clip=(x0, y0, x1, y1))
            print(f'model: {model_map} {"pdf_pesquisavel" if pdf_pesquisavel_map else "raster_pdf"}: labe: {label:>30} || x0:{x0:>6} | y0:{y0:>6} | x1:{x1:>6} | y1:{y1:>6}\n {text}')
            valor_total_match = re.search(r'R\$ ([\d,.]+)', text)
            if valor_total_match:
                valor_total_sem_formatacao = valor_total_match.group(1).replace('.', '').replace(',', '.')
                valor_total_float = float(valor_total_sem_formatacao)
                #nf_data_valor_total['valor_total_nota'] =
                
    pdf_document.close()            
                
    return valor_total_float  
   nf_data_CNAE = {}
    nf_data_CNAE['Secao'] = "6. CNAE e Item da Lista de Serviços"
    
    process = ["cnae", 'item_lista_servicos']
    
    section = "6. CNAE e Item da Lista de Serviços"
    pdf_pesquisavel_map = True
        
    pdf_document = fitz.open(file_path)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]    
        
    tipo = "sframe_field"
    message_erro = []
    nf_dados_prestador = {}
    # model = row['model']
    for father in process:
        label = father
        if label == "cnae": 
            tipo = "sframe_field"
            print(model_map)
            coordinates = get_coordinates_filter(pdf_pesquisavel_map=pdf_pesquisavel_map, model=model_map, tipo=tipo, label=label, section=section)
            x0, y0, x1, y1 = coordinates[0]
            text = page.get_text("text", clip=(x0, y0, x1, y1))
            print(f'model: {model_map} {"pdf_pesquisavel" if pdf_pesquisavel_map else "raster_pdf"}: labe: {label:>30} || x0:{x0:>6} | y0:{y0:>6} | x1:{x1:>6} | y1:{y1:>6}\n {text}')
        
        elif label == "item_lista_servicos":
            tipo = "sframe_field"
            print(model_map)
            coordinates = get_coordinates_filter(pdf_pesquisavel_map=pdf_pesquisavel_map, model=model_map, tipo=tipo, label=label, section=section)
            x0, y0, x1, y1 = coordinates[0]
            text = page.get_text("text", clip=(x0, y0, x1, y1))
            print(f'model: {model_map} {"pdf_pesquisavel" if pdf_pesquisavel_map else "raster_pdf"}: labe: {label:>30} || x0:{x0:>6} | y0:{y0:>6} | x1:{x1:>6} | y1:{y1:>6}\n {text}')
                

           